# Generate routes

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn.cluster import KMeans

In [4]:
data_stops = pd.read_csv('data/uni_molde_v3.csv', sep=';')

data_stops
data_stops.loc[data_stops['stop_completed_at'].isna(), 'stop_completed_at'] = "-1"
data_stops.loc[data_stops['stop_arrived_at'].isna(), 'stop_arrived_at'] = "-1"

print(data_stops.isnull().sum())


sorted_data_stops = data_stops.sort_values(by='stop_dispatched_at', ascending=True)
sorted_data_stops = sorted_data_stops.reset_index(drop=True)
sorted_data_stops['day_of_week'] = pd.to_datetime(sorted_data_stops['stop_dispatched_at']).dt.day_name()
sorted_data_stops['date'] = pd.to_datetime(sorted_data_stops['stop_dispatched_at']).dt.date

#clustering
locations_df = sorted_data_stops[['current_lat', 'current_lng']]
kmeans = KMeans(n_clusters=5000, random_state=42)
kmeans.fit(locations_df)
sorted_data_stops['cluster'] = kmeans.labels_ + 1

sorted_data_stops['location_id_craft'] = sorted_data_stops.groupby(['current_lat', 'current_lng']).ngroup()+1
with open('output.txt', 'w') as f:
    print(sorted_data_stops.to_string(), file=f)
print('number of groups', sorted_data_stops['location_id_craft'].nunique())

# data_stops_day= sorted_data_stops[sorted_data_stops['day_of_week'] == "Wednesday"]
grouped_df = sorted_data_stops.groupby('driver_workday_id')[['driver_id', 'location_type_id', 'address_id', 'location_id','stop_dispatched_at', 'stop_arrived_at', 'stop_earliest', 'stop_latest', 'current_lat', 'current_lng', 'stop_completed_at', 'cluster', 'location_id_craft','day_of_week', 'date', 'location_is_depot']].apply(lambda x: pd.Series({
    'driver_id': x['driver_id'].tolist(),
    'location_type_id': x['location_type_id'].tolist(),
    'planned_route_location': x['location_id'].tolist(),
    'stop_dispatched_at': x['stop_dispatched_at'].tolist(),
    'stop_arrived_at': x['stop_arrived_at'].tolist(),
    'stop_earliest': x['stop_earliest'].tolist(),
    'stop_latest': x['stop_latest'].tolist(),
    'current_lat': x['current_lat'].tolist(),
    'current_lng': x['current_lng'].tolist(),
    'stop_completed_at': x['stop_completed_at'].tolist(),
    'planned_route_cluster': x['cluster'].tolist(),
    'planned_route_craft': x['location_id_craft'].tolist(),
    'day_of_week': x['day_of_week'].tolist(),
    'location_is_depot': x['location_is_depot'].tolist(),
    'date': x['date'].tolist()
})).reset_index()
grouped_df
# grouped_df

stop_completed_at         0
stop_arrived_at           0
stop_latest               0
stop_earliest             0
stop_dispatched_at        0
location_id               0
location_type_id          0
driver_workday_id         0
address_id                0
location_is_depot         0
driver_id                 0
current_lat               0
current_lng               0
prev_planned_lat      27440
prev_planned_lng      27440
prev_actual_lat       27440
prev_actual_lng       27440
dtype: int64
number of groups 11589


,driver_workday_id,driver_id,location_type_id,planned_route_location,stop_dispatched_at,stop_arrived_at,stop_earliest,stop_latest,current_lat,current_lng,stop_completed_at,planned_route_cluster,planned_route_craft,day_of_week,location_is_depot,date
0,297900,"[1255, 1255, 1255, 1255, 1255, 1255, 1255, 125...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2]","[132992, 128289, 128104, 128048, 127766, 12845...","[2024-01-01 19:41:35.136881+00, 2024-01-01 19:...","[2024-01-01 22:14:16.454324+00, 2024-01-01 23:...","[2024-01-01 19:00:00+00, 2024-01-01 23:00:00+0...","[2024-01-02 01:00:00+00, 2024-01-02 03:00:00+0...","[55.6465393, 55.6669147, 55.676665, 55.6757322...","[12.1522978, 12.5262801, 12.566428, 12.5619711...","[2024-01-01 22:14:54.872731+00, 2024-01-01 23:...","[3165, 1419, 3238, 3429, 720, 1360, 4511, 3703...","[1169, 1405, 1577, 1560, 1479, 1391, 1434, 141...","[Monday, Monday, Monday, Monday, Monday, Monda...","[True, False, False, False, False, False, Fals...","[2024-01-01, 2024-01-01, 2024-01-01, 2024-01-0..."
1,297906,"[1195, 1195, 1195, 1195, 1195, 1195, 1195]","[1, 2, 2, 2, 2, 2, 2]","[132992, 130073, 130075, 128234, 130076, 14676...","[2024-01-01 19:42:05.147145+00, 2024-01-01 19:...","[2024-01-01 20:50:17.003744+00, 2024-01-02 02:...","[2024-01-01 19:00:00+00, 2024-01-01 21:00:00+0...","[2024-01-02 01:00:00+00, 2024-01-02 04:00:00+0...","[55.6465393, 55.662018, 55.662429, 55.6727433,...","[12.1522978, 12.360745, 12.363469, 12.3935111,...","[2024-01-01 20:50:29.721122+00, 2024-01-02 02:...","[3165, 122, 3540, 1413, 3487, 2070, 1287]","[1169, 1352, 1356, 1508, 1350, 1312, 1401]","[Monday, Monday, Monday, Monday, Monday, Monda...","[True, False, False, False, False, False, False]","[2024-01-01, 2024-01-01, 2024-01-01, 2024-01-0..."
2,297908,"[1176, 1176, 1176, 1176, 1176, 1176, 1176, 117...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[132992, 130448, 128643, 128641, 128584, 13017...","[2024-01-01 21:25:47.230131+00, 2024-01-01 21:...","[2024-01-01 23:06:27.627368+00, 2024-01-02 01:...","[2024-01-01 19:00:00+00, 2024-01-01 21:00:00+0...","[2024-01-02 01:00:00+00, 2024-01-02 04:00:00+0...","[55.6465393, 55.619592, 55.636304, 55.631888, ...","[12.1522978, 12.028735, 12.044857, 12.055067, ...","[2024-01-01 23:06:36.064172+00, 2024-01-02 01:...","[3165, 1073, 1995, 3776, 1059, 1266, 1938, 193...","[1169, 960, 1049, 1026, 982, 916, 1284, 1281, ...","[Monday, Monday, Monday, Monday, Monday, Monda...","[True, False, False, False, False, False, Fals...","[2024-01-01, 2024-01-01, 2024-01-01, 2024-01-0..."
3,297911,"[1197, 1197, 1197, 1197, 1197, 1197, 1197, 119...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2]","[132992, 153283, 146745, 146743, 146740, 14673...","[2024-01-01 19:42:59.704147+00, 2024-01-01 19:...","[2024-01-02 00:09:15.241119+00, 2024-01-02 04:...","[2024-01-02 00:00:00+00, 2024-01-02 04:30:00+0...","[2024-01-02 22:00:00+00, 2024-01-02 08:00:00+0...","[55.6465393, 54.65117396302384, 54.666191, 54....","[12.1522978, 11.375901220879824, 11.333044, 11...","[2024-01-02 00:09:21.131516+00, 2024-01-02 04:...","[3165, 442, 137, 137, 137, 137, 137, 137, 1809...","[1169, 7, 23, 23, 23, 23, 23, 23, 12, 4]","[Monday, Monday, Monday, Monday, Monday, Monda...","[True, False, False, False, False, False, Fals...","[2024-01-01, 2024-01-01, 2024-01-01, 2024-01-0..."
4,297919,"[1218, 1218, 1218, 1218, 1218, 1218, 1218, 121...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]","[132992, 129015, 127984, 128413, 128017, 12811...","[2024-01-01 19:46:51.16173+00, 2024-01-01 19:4...","[2024-01-01 23:05:52.039818+00, 2024-01-02 01:...","[2024-01-01 19:00:00+00, 2024-01-01 21:00:00+0...","[2024-01-02 01:00:00+00, 2024-01-02 04:00:00+0...","[55.6465393, 55.62296, 55.562168, 55.6381577, ...","[12.1522978, 12.615835, 12.576841, 12.6515491,...","[2024-01-01 23:05:58.283704+00, 2024-01-02 01:...","[3165, 3301, 537, 1830, 4879, 3305, 3305, 1642...","[1169, 965, 818, 1070, 1658, 1168, 1166, 888, ...","[Monday, Monday, Monday, Monday, Monday, Monda...","[True, False, False, Fa

In [5]:
sorted_data_stops.head()

,stop_completed_at,stop_arrived_at,stop_latest,stop_earliest,stop_dispatched_at,location_id,location_type_id,driver_workday_id,address_id,location_is_depot,...,current_lat,current_lng,prev_planned_lat,prev_planned_lng,prev_actual_lat,prev_actual_lng,day_of_week,date,cluster,location_id_craft
0,2024-01-02 11:10:52.185782+00,2024-01-02 11:10:51.209288+00,2024-01-02 22:59:00+00,2024-01-02 05:00:00+00,2023-12-20 09:20:12.139397+00,104448,1,299448,80079,False,...,56.455617,10.051820,NaN,NaN,NaN,NaN,Wednesday,2023-12-20,1499,4134
1,2024-01-02 11:10:52.185783+00,2024-01-02 11:10:51.209289+00,2024-01-02 22:59:00+00,2024-01-02 05:00:00+00,2023-12-20 09:20:12.144071+00,86931,2,299448,81843,False,...,59.473000,10.318318,56.455617,10.051820,56.455617,10.051820,Wednesday,2023-12-20,684,5561
2,2024-01-02 11:10:52.178147+00,2024-01-02 11:10:51.03874+00,2024-01-02 21:00:00+00,2024-01-02 05:00:00+00,2023-12-20 09:21:01.657773+00,86931,1,299447,81843,False,...,59.473000,10.318318,NaN,NaN,NaN,NaN,Wednesday,2023-12-20,684,5561
3,2024-01-02 11:10:52.178148+00,2024-01-02 11:10:51.038741+00,2024-01-02 21:00:00+00,2024-01-02 05:00:00+00,2023-12-20 09:21:01.661233+00,148025,2,299447,134404,False,...,58.783764,5.734901,59.473000,10.318318,59.473000,10.318318,Wednesday,2023-12-20,55,4265
4,2024-01-02 08:39:35.128145+00,2024-01-02 07:45:25.310456+00,2024-01-02 21:00:00+00,2024-01-02 05:00:00+00,2023-12-20 09:22:29.569837+00,86931,1,299444,81843,False,...,59.473000,10.318318,NaN,NaN,NaN,NaN,Wednesday,2023-12-20,684,5561


In [6]:
from geopy.distance import geodesic
import pandas as pd

# Function to calculate distance between two coordinates
def calculate_distance(row):
    distances = []
    for i in range(len(row['planned_route_craft'])-1):
        coords_1 = (sorted_data_stops.loc[sorted_data_stops['location_id_craft'] == row['planned_route_craft'][i], ['current_lat']].values[0][0],
                     sorted_data_stops.loc[sorted_data_stops['location_id_craft'] == row['planned_route_craft'][i], ['current_lng']].values[0][0])
        coords_2 = (sorted_data_stops.loc[sorted_data_stops['location_id_craft'] == row['planned_route_craft'][i+1], ['current_lat']].values[0][0],
                     sorted_data_stops.loc[sorted_data_stops['location_id_craft'] == row['planned_route_craft'][i+1], ['current_lng']].values[0][0])
        distances.append(geodesic(coords_1, coords_2).miles)
    return distances

def calculate_distance_actual(row):
    distances = []
    for i in range(len(row['actual_route_location'])-1):
        coords_1 = (sorted_data_stops.loc[sorted_data_stops['location_id_craft'] == row['actual_route_location'][i], ['current_lat']].values[0][0],
                     sorted_data_stops.loc[sorted_data_stops['location_id_craft'] == row['actual_route_location'][i], ['current_lng']].values[0][0])
        coords_2 = (sorted_data_stops.loc[sorted_data_stops['location_id_craft'] == row['actual_route_location'][i+1], ['current_lat']].values[0][0],
                     sorted_data_stops.loc[sorted_data_stops['location_id_craft'] == row['actual_route_location'][i+1], ['current_lng']].values[0][0])
        distances.append(geodesic(coords_1, coords_2).miles)
    return distances

# Create a new column 'distance_route' in 'final_routes'
grouped_df['distance_route'] = grouped_df.apply(calculate_distance, axis=1)
# total_sum_distance = grouped_df['distance_route'].sum()
# grouped_df['total_distance'] = total_sum_distance

In [7]:
grouped_df['total_distance_planned'] = grouped_df['distance_route'].apply(sum)
grouped_df


,driver_workday_id,driver_id,location_type_id,planned_route_location,stop_dispatched_at,stop_arrived_at,stop_earliest,stop_latest,current_lat,current_lng,stop_completed_at,planned_route_cluster,planned_route_craft,day_of_week,location_is_depot,date,distance_route,total_distance_planned
0,297900,"[1255, 1255, 1255, 1255, 1255, 1255, 1255, 125...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2]","[132992, 128289, 128104, 128048, 127766, 12845...","[2024-01-01 19:41:35.136881+00, 2024-01-01 19:...","[2024-01-01 22:14:16.454324+00, 2024-01-01 23:...","[2024-01-01 19:00:00+00, 2024-01-01 23:00:00+0...","[2024-01-02 01:00:00+00, 2024-01-02 03:00:00+0...","[55.6465393, 55.6669147, 55.676665, 55.6757322...","[12.1522978, 12.5262801, 12.566428, 12.5619711...","[2024-01-01 22:14:54.872731+00, 2024-01-01 23:...","[3165, 1419, 3238, 3429, 720, 1360, 4511, 3703...","[1169, 1405, 1577, 1560, 1479, 1391, 1434, 141...","[Monday, Monday, Monday, Monday, Monday, Monda...","[True, False, False, False, False, False, Fals...","[2024-01-01, 2024-01-01, 2024-01-01, 2024-01-0...","[14.694940475474436, 1.7084597707961175, 0.185...",19.039848
1,297906,"[1195, 1195, 1195, 1195, 1195, 1195, 1195]","[1, 2, 2, 2, 2, 2, 2]","[132992, 130073, 130075, 128234, 130076, 14676...","[2024-01-01 19:42:05.147145+00, 2024-01-01 19:...","[2024-01-01 20:50:17.003744+00, 2024-01-02 02:...","[2024-01-01 19:00:00+00, 2024-01-01 21:00:00+0...","[2024-01-02 01:00:00+00, 2024-01-02 04:00:00+0...","[55.6465393, 55.662018, 55.662429, 55.6727433,...","[12.1522978, 12.360745, 12.363469, 12.3935111,...","[2024-01-01 20:50:29.721122+00, 2024-01-02 02:...","[3165, 122, 3540, 1413, 3487, 2070, 1287]","[1169, 1352, 1356, 1508, 1350, 1312, 1401]","[Monday, Monday, Monday, Monday, Monday, Monda...","[True, False, False, False, False, False, False]","[2024-01-01, 2024-01-01, 2024-01-01, 2024-01-0...","[8.223311758249285, 0.11025550750368773, 1.374...",12.124804
2,297908,"[1176, 1176, 1176, 1176, 1176, 1176, 1176, 117...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[132992, 130448, 128643, 128641, 128584, 13017...","[2024-01-01 21:25:47.230131+00, 2024-01-01 21:...","[2024-01-01 23:06:27.627368+00, 2024-01-02 01:...","[2024-01-01 19:00:00+00, 2024-01-01 21:00:00+0...","[2024-01-02 01:00:00+00, 2024-01-02 04:00:00+0...","[55.6465393, 55.619592, 55.636304, 55.631888, ...","[12.1522978, 12.028735, 12.044857, 12.055067, ...","[2024-01-01 23:06:36.064172+00, 2024-01-02 01:...","[3165, 1073, 1995, 3776, 1059, 1266, 1938, 193...","[1169, 960, 1049, 1026, 982, 916, 1284, 1281, ...","[Monday, Monday, Monday, Monday, Monday, Monda...","[True, False, False, False, False, False, Fals...","[2024-01-01, 2024-01-01, 2024-01-01, 2024-01-0...","[5.182600931419761, 1.3171408361998556, 0.5029...",56.772919
3,297911,"[1197, 1197, 1197, 1197, 1197, 1197, 1197, 119...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2]","[132992, 153283, 146745, 146743, 146740, 14673...","[2024-01-01 19:42:59.704147+00, 2024-01-01 19:...","[2024-01-02 00:09:15.241119+00, 2024-01-02 04:...","[2024-01-02 00:00:00+00, 2024-01-02 04:30:00+0...","[2024-01-02 22:00:00+00, 2024-01-02 08:00:00+0...","[55.6465393, 54.65117396302384, 54.666191, 54....","[12.1522978, 11.375901220879824, 11.333044, 11...","[2024-01-02 00:09:21.131516+00, 2024-01-02 04:...","[3165, 442, 137, 137, 137, 137, 137, 137, 1809...","[1169, 7, 23, 23, 23, 23, 23, 23, 12, 4]","[Monday, Monday, Monday, Monday, Monday, Monda...","[True, False, False, False, False, False, Fals...","[2024-01-01, 2024-01-01, 2024-01-01, 2024-01-0...","[75.41095073521456, 2.0081266994719744, 0.0, 0...",79.685240
4,297919,"[1218, 1218, 1218, 1218, 1218, 1218, 1218, 121...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]","[132992, 129015, 127984, 128413, 128017, 12811...","[2024-01-01 19:46:51.16173+00, 2024-01-01 19:4...","[2024-01-01 23:05:52.039818+00, 2024-01-02 01:...","[2024-01-01 19:00:00+00, 2024-01-01 21:00:00+0...","[2024-01-02 01:00:00+00, 2024-01-02 04:00:00+0...","[55.6465393, 55.62296, 55.562168, 55.6381577, ...

In [8]:
grouped_df['total_distance_planned'] = grouped_df['distance_route'].apply(sum)
routes = grouped_df[grouped_df.apply(lambda row: max(row['stop_dispatched_at']) < min(row['stop_completed_at']), axis=1)]
routes = routes.reset_index(drop=True)
len(routes)

20692

In [9]:
index_routes_with_na = []
for i in range(len(routes)):
    row = routes.iloc[i]
    if "-1" in row['stop_arrived_at']:
        index_routes_with_na.append(i)
print("The number of routes where one value is NA(arrived time)", len(index_routes_with_na))
print(index_routes_with_na)
routes = routes.drop(index_routes_with_na)
routes.reset_index(drop=True)
#
print(len(routes))

The number of routes where one value is NA(arrived time) 23
[1187, 1464, 1688, 1746, 2113, 3424, 3880, 4471, 4974, 5098, 5172, 5182, 5540, 5787, 6476, 7626, 10793, 11161, 11621, 13845, 17739, 19786, 20033]
20669


In [10]:
#check if it is the same driver in the route
routes['day_of_week'] = routes['stop_arrived_at'].apply(lambda x: [pd.to_datetime(dt).day_name() for dt in x])

def get_mode(x):
    return pd.Series(x).mode().iloc[0] if not pd.Series(x).mode().empty else np.nan

for i in routes['driver_id']:
    if not all(x == i[0] for x in i):
            print('Not the same driver in the route')
routes['driver_id'] = routes['driver_id'].apply(lambda x : x[0])

routes['day_of_week'] = routes['day_of_week'].apply(get_mode)
routes['date'] = routes['date'].apply(get_mode)

routes['date'] = pd.to_datetime(routes['date'])
routes['last_two_weeks_count'] = routes.apply(lambda row:
                                      routes[(routes['driver_id'] == row['driver_id']) &
                                         (row['date'] - routes['date']).dt.days.between(-14, 0)].shape[0],
                                      axis=1)
routes = routes[(routes['date'] < '2023-12-21') | (routes['date'] > '2024-01-14')]
routes['location_type_id'] = routes['location_type_id'].apply(lambda x: [0 if i == 1 else 1 for i in x])
routes['location_is_depot'] = routes['location_is_depot'].apply(lambda x: [int(value) for value in x])

routes = routes[routes['planned_route_craft'].apply(lambda x: len(x) > 2)]
routes = routes.reset_index(drop=True)

In [11]:
routes

,driver_workday_id,driver_id,location_type_id,planned_route_location,stop_dispatched_at,stop_arrived_at,stop_earliest,stop_latest,current_lat,current_lng,stop_completed_at,planned_route_cluster,planned_route_craft,day_of_week,location_is_depot,date,distance_route,total_distance_planned,last_two_weeks_count
0,305735,1255,"[0, 1, 1, 1, 1, 1, 1, 1]","[132992, 128289, 128048, 128448, 128037, 12845...","[2024-01-15 20:13:10.25734+00, 2024-01-15 20:1...","[2024-01-15 22:11:49.528537+00, 2024-01-15 23:...","[2024-01-15 19:00:00+00, 2024-01-15 23:00:00+0...","[2024-01-16 01:00:00+00, 2024-01-16 03:00:00+0...","[55.64641920439919, 55.6669147, 55.6757322, 55...","[12.15372974969827, 12.5262801, 12.5619711, 12...","[2024-01-15 22:14:55.135437+00, 2024-01-15 23:...","[147, 1419, 3429, 3703, 1360, 1360, 4511, 335]","[1167, 1405, 1560, 1413, 1391, 1391, 1434, 1269]",Tuesday,"[1, 0, 0, 0, 0, 0, 0, 0]",2024-01-15,"[14.640018023192697, 1.5229312527257717, 0.606...",18.615111,8
1,305741,1195,"[0, 1, 1, 1, 1, 1, 1, 1]","[132992, 130073, 146768, 130075, 128234, 12823...","[2024-01-15 20:14:10.430711+00, 2024-01-15 20:...","[2024-01-15 22:58:58.038139+00, 2024-01-15 23:...","[2024-01-15 19:00:00+00, 2024-01-15 21:00:00+0...","[2024-01-16 01:00:00+00, 2024-01-16 04:00:00+0...","[55.64641920439919, 55.662018, 55.658103, 55.6...","[12.15372974969827, 12.360745, 12.368506, 12.3...","[2024-01-15 22:59:10.867219+00, 2024-01-15 23:...","[147, 122, 2070, 3540, 1413, 1413, 3663, 1287]","[1167, 1352, 1312, 1356, 1508, 1508, 1621, 1401]",Monday,"[1, 0, 0, 0, 0, 0, 0, 0]",2024-01-15,"[8.16888775040224, 0.4067943329231743, 0.35828...",12.682013,11
2,305742,1186,"[0, 1, 1, 1, 1, 1, 1]","[132992, 128251, 128480, 129701, 130263, 13014...","[2024-01-15 20:15:12.147596+00, 2024-01-15 20:...","[2024-01-15 22:32:42.615543+00, 2024-01-15 23:...","[2024-01-15 19:00:00+00, 2024-01-15 21:00:00+0...","[2024-01-16 01:00:00+00, 2024-01-16 04:00:00+0...","[55.64641920439919, 55.7498333, 55.80148, 55.7...","[12.15372974969827, 12.5870785, 12.525814, 12....","[2024-01-15 22:32:54.670991+00, 2024-01-15 23:...","[147, 2596, 521, 4623, 3326, 3115, 1050]","[1167, 2231, 2406, 2349, 2298, 2588, 2493]",Tuesday,"[1, 0, 0, 0, 0, 0, 0]",2024-01-15,"[18.380690356980487, 4.298068156860068, 1.5617...",34.403233,9
3,305754,1218,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[132992, 128114, 129560, 128017, 128413, 12798...","[2024-01-15 20:17:12.871538+00, 2024-01-15 20:...","[2024-01-15 22:37:07.860851+00, 2024-01-16 00:...","[2024-01-15 19:00:00+00, 2024-01-15 21:00:00+0...","[2024-01-16 01:00:00+00, 2024-01-16 04:00:00+0...","[55.64641920439919, 55.646535, 55.672309, 55.6...","[12.15372974969827, 12.539958, 12.588959, 12.6...","[2024-01-15 22:37:19.03627+00, 2024-01-16 00:2...","[147, 3305, 3715, 4879, 1830, 537, 3301, 2199,...","[1167, 1168, 1495, 1658, 1070, 818, 965, 936, ...",Tuesday,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",2024-01-15,"[15.110089331615377, 2.617611669361755, 0.8878...",53.800967,11
4,305758,1192,"[0, 1, 1, 1, 1, 1, 1, 1, 1]","[132992, 127628, 128567, 128593, 127977, 12878...","[2024-01-15 20:17:36.851051+00, 2024-01-15 20:...","[2024-01-15 23:06:33.822316+00, 2024-01-16 01:...","[2024-01-15 19:00:00+00, 2024-01-15 21:00:00+0...","[2024-01-16 01:00:00+00, 2024-01-16 04:00:00+0...","[55.64641920439919, 55.640375, 55.639120600000...","[12.15372974969827, 12.091312, 12.0627714, 12....","[2024-01-15 23:06:46.892607+00, 2024-01-16 01:...","[147, 4979, 3980, 3679, 2681, 2319, 708, 1644,...","[1167, 1101, 1079, 1282, 1123, 1098, 834, 1262...",Tuesday,"[1, 0, 0, 0, 0, 0, 0, 0, 0]",2024-01-15,"[2.4776512575967895, 1.1201296974431052, 1.598...",18.376147,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18557,442390,276,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[23287, 24605, 171404, 62110, 25046, 151521, 3...","[2024-08-06 19:39:14.014177+00, 2024-08-06 19:...","[2024-08-07 13:31:52.955936+00, 2024-08-07 13:...","[2024-08-07 01:00:00+00, 2024-08-07 05:00:00+

In [12]:
def create_actual_route(df, column):
    res_col = []
    for i in range(len(df)):
        row = df.iloc[i]
        sorted_dates = sorted(row['stop_arrived_at'])
        mapping = {}
        for i, date in enumerate(sorted_dates):
            mapping[date] = row[column][i]
        res_val = [mapping[row['stop_arrived_at'][i]] for i in range(len(row['stop_arrived_at']))]
        res_col.append(res_val)
    return res_col

def create_actual_route_type(df, column):
    res_col = []
    for i in range(len(df)):
        row = df.iloc[i]
        sorted_dates = sorted(row['stop_arrived_at'])
        mapping = {}
        for i, date in enumerate(sorted_dates):
            mapping[date] = row['location_type_id'][i]
        res_val = [mapping[row['stop_arrived_at'][i]] for i in range(len(row['stop_arrived_at']))]
        res_col.append(res_val)
    return res_col

routes['actual_route_location'] = create_actual_route(routes, 'planned_route_craft')
routes['actual_route_type'] = create_actual_route_type(routes, 'planned_route_craft')
routes['distance_actual_route'] = routes.apply(calculate_distance_actual, axis=1)
routes['total_distance_actual'] = routes['distance_actual_route'].apply(sum)
routes['difference_distance'] = (routes['total_distance_actual'] - routes['total_distance_planned'])/routes['total_distance_planned']

#remove all the routes with length <= 2
routes = routes[routes['planned_route_craft'].apply(lambda x: len(x) > 2)]
routes = routes.reset_index(drop=True)

len(routes)

18562

In [13]:
routes = routes.reset_index(drop=True)
more_than_one_1 = routes[routes['location_is_depot'].apply(lambda x: x.count(True) > 1)]
len(more_than_one_1), len(routes)

routes['actual_route_unique'] = create_actual_route(routes, 'planned_route_location')

In [14]:
planned_routes = routes[['planned_route_craft', 'actual_route_location', 'driver_id', 'day_of_week', 'last_two_weeks_count', 'location_type_id', 'stop_arrived_at', 'stop_earliest', 'stop_latest', 'current_lat', 'current_lng', 'difference_distance', 'distance_route', 'distance_actual_route', 'location_is_depot', 'location_type_id', 'date', 'planned_route_location', 'actual_route_unique']]
actual_routes = routes['actual_route_location']

In [15]:
planned_routes

,planned_route_craft,actual_route_location,driver_id,day_of_week,last_two_weeks_count,location_type_id,stop_arrived_at,stop_earliest,stop_latest,current_lat,current_lng,difference_distance,distance_route,distance_actual_route,location_is_depot,location_type_id,date,planned_route_location,actual_route_unique
0,"[1167, 1405, 1560, 1413, 1391, 1391, 1434, 1269]","[1167, 1405, 1560, 1413, 1391, 1391, 1434, 1269]",1255,Tuesday,8,"[0, 1, 1, 1, 1, 1, 1, 1]","[2024-01-15 22:11:49.528537+00, 2024-01-15 23:...","[2024-01-15 19:00:00+00, 2024-01-15 23:00:00+0...","[2024-01-16 01:00:00+00, 2024-01-16 03:00:00+0...","[55.64641920439919, 55.6669147, 55.6757322, 55...","[12.15372974969827, 12.5262801, 12.5619711, 12...",0.000000,"[14.640018023192697, 1.5229312527257717, 0.606...","[14.640018023192697, 1.5229312527257717, 0.606...","[1, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 1, 1, 1, 1, 1, 1]",2024-01-15,"[132992, 128289, 128048, 128448, 128037, 12845...","[132992, 128289, 128048, 128448, 128454, 12803..."
1,"[1167, 1352, 1312, 1356, 1508, 1508, 1621, 1401]","[1167, 1312, 1356, 1352, 1508, 1508, 1621, 1401]",1195,Monday,11,"[0, 1, 1, 1, 1, 1, 1, 1]","[2024-01-15 22:58:58.038139+00, 2024-01-15 23:...","[2024-01-15 19:00:00+00, 2024-01-15 21:00:00+0...","[2024-01-16 01:00:00+00, 2024-01-16 04:00:00+0...","[55.64641920439919, 55.662018, 55.658103, 55.6...","[12.15372974969827, 12.360745, 12.368506, 12.3...",0.006369,"[8.16888775040224, 0.4067943329231743, 0.35828...","[8.440075587409163, 0.35828887642036544, 0.110...","[1, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 1, 1, 1, 1, 1, 1]",2024-01-15,"[132992, 130073, 146768, 130075, 128234, 12823...","[132992, 146768, 130075, 130073, 128234, 12823..."
2,"[1167, 2231, 2406, 2349, 2298, 2588, 2493]","[1167, 2231, 2349, 2406, 2298, 2493, 2588]",1186,Tuesday,9,"[0, 1, 1, 1, 1, 1, 1]","[2024-01-15 22:32:42.615543+00, 2024-01-15 23:...","[2024-01-15 19:00:00+00, 2024-01-15 21:00:00+0...","[2024-01-16 01:00:00+00, 2024-01-16 04:00:00+0...","[55.64641920439919, 55.7498333, 55.80148, 55.7...","[12.15372974969827, 12.5870785, 12.525814, 12....",-0.027603,"[18.380690356980487, 4.298068156860068, 1.5617...","[18.380690356980487, 3.164379243258826, 1.5617...","[1, 0, 0, 0, 0, 0, 0]","[0, 1, 1, 1, 1, 1, 1]",2024-01-15,"[132992, 128251, 128480, 129701, 130263, 13014...","[132992, 128251, 129701, 128480, 130263, 14652..."
3,"[1167, 1168, 1495, 1658, 1070, 818, 965, 936, ...","[1167, 1070, 936, 822, 963, 965, 818, 1658, 14...",1218,Tuesday,11,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[2024-01-15 22:37:07.860851+00, 2024-01-16 00:...","[2024-01-15 19:00:00+00, 2024-01-15 21:00:00+0...","[2024-01-16 01:00:00+00, 2024-01-16 04:00:00+0...","[55.64641920439919, 55.646535, 55.672309, 55.6...","[12.15372974969827, 12.539958, 12.588959, 12.6...",0.435438,"[15.110089331615377, 2.617611669361755, 0.8878...","[19.4862183744275, 11.162649116186252, 5.20670...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1]",2024-01-15,"[132992, 128114, 129560, 128017, 128413, 12798...","[132992, 128413, 128394, 129091, 128617, 12901..."
4,"[1167, 1101, 1079, 1282, 1123, 1098, 834, 1262...","[1167, 1098, 1315, 1282, 1262, 834, 1123, 1079...",1192,Tuesday,11,"[0, 1, 1, 1, 1, 1, 1, 1, 1]","[2024-01-15 23:06:33.822316+00, 2024-01-16 01:...","[2024-01-15 19:00:00+00, 2024-01-15 21:00:00+0...","[2024-01-16 01:00:00+00, 2024-01-16 04:00:00+0...","[55.64641920439919, 55.640375, 55.639120600000...","[12.15372974969827, 12.091312, 12.0627714, 12....",0.162129,"[2.4776512575967895, 1.1201296974431052, 1.598...","[2.5909389807670826, 2.6002283089649945, 2.104...","[1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 1, 1, 1, 1, 1, 1, 1]",2024-01-15,"[132992, 127628, 128567, 128593, 127977, 12878...","[132992, 128788, 128647, 128593, 128612, 12860..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18557,"[10076, 10099, 9596, 9507, 9158, 8853, 9640, 8...","[10076, 10099, 9596, 9507, 9158, 8853, 9640, 8...",276,Wednesday,1,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [16]:
planned_routes_list = planned_routes['planned_route_craft'].tolist()
actual_routes_list = actual_routes.tolist()

# filtered_rows = []
#
# for row in actual_routes_list:
#     if row not in planned_routes_list:
#         filtered_rows.append(row)

#duplicated removed
# planned_routes_list = [array for i, array in enumerate(planned_routes_list) if array not in planned_routes_list[:i]]
# actual_routes_list = [array for i, array in enumerate(filtered_rows) if array not in filtered_rows[:i]]
len(planned_routes_list),len(actual_routes_list)

(18562, 18562)

# Edit distance

In [17]:
# Edit Distance

def minDistance(word1, word2) -> int:
    m = len(word1)
    n = len(word2)
    # dp[i][j] := min # Of operations to convert word1[0..i) to word2[0..j)
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    for i in range(1, m + 1):
      dp[i][0] = i

    for j in range(1, n + 1):
      dp[0][j] = j

    for i in range(1, m + 1):
      for j in range(1, n + 1):
        if word1[i - 1] == word2[j - 1]:
          dp[i][j] = dp[i - 1][j - 1]
        else:
          dp[i][j] = min(dp[i - 1][j - 1], dp[i - 1][j], dp[i][j - 1]) + 1

    return dp[m][n]

print(minDistance([878, 1526, 1613, 1706, 1857, 1777], [878, 1526, 1706, 1613, 1777, 1857]))

3



# Route quality score

In [18]:
def calculate_route_quality_score(planned_ranks, actual_ranks):
    # Calculate sum of absolute differences in ranks
    sum_of_differences = sum(abs(actual_ranks.index(x) - planned_ranks.index(x)) for x in planned_ranks)

    # Calculate max possible difference
    max_possible_difference = sum([abs(2 * i - (len(planned_ranks) + 1)) for i in range(1, len(planned_ranks) + 1)])

    # Calculate route quality score
    route_quality_score = 1 - (sum_of_differences / max_possible_difference)

    return route_quality_score

For each driver-day, we consider only the planned route. For each planned route we calculate how much it deviates from the actual route (e.g., use some form of edit distance - see https://link.springer.com/article/10.1007/s10732-006-9001-3?), which is then normalized (divide by max edit distance, or number of visits or something?) to the interval [0, 1]. Then, a planned route is good if this distance (between planned and actual) is less than a certain threshold and otherwise it is bad. This becomes the label (bad/good)

In [19]:
import statistics
completed_routes_df = pd.DataFrame(columns=['planned_route_craft', 'actual_route_location', 'driver_id', 'day_of_week', 'distance_route', 'distance_actual_route', 'last_two_weeks_count', 'stop_arrived_at', 'stop_earliest', 'stop_latest', 'current_lat', 'current_lng', 'location_type_id', 'location_is_depot', 'date','actual_route_unique','planned_route_location'])
uncompleted_routes_df = pd.DataFrame(columns=['planned_route_craft', 'actual_route_location', 'driver_id', 'day_of_week', 'distance_route', 'distance_actual_route', 'last_two_weeks_count', 'stop_arrived_at', 'stop_earliest', 'stop_latest', 'current_lat', 'current_lng', 'location_type_id', 'location_is_depot', 'date', 'actual_route_unique', 'planned_route_location'])
#11700795
scores = []
edit_distances = []
completed_routes_list = []
uncompleted_routes_list = []
for i in range(len(planned_routes_list)):
    editDistance = minDistance(planned_routes_list[i], actual_routes_list[i]) / len(planned_routes_list[i])
    score = calculate_route_quality_score(planned_routes_list[i], actual_routes_list[i])
    scores.append(score)
    edit_distances.append(editDistance)

    # if editDistance > 0.05 and score < 0.95 and planned_routes['difference_distance'].iloc[i] > 0.01:
    if editDistance <= 0.3 and score >= 0.7 and planned_routes['difference_distance'].iloc[i] <= 0.3:
    # if editDistance <= 0 and score >= 1 and planned_routes['difference_distance'].iloc[i] <= 0:
        completed_routes_list.append(planned_routes.iloc[i].to_dict())
    else:
        uncompleted_routes_list.append(planned_routes.iloc[i].to_dict())

# Convert the lists to DataFrames
completed_routes_df = pd.DataFrame(completed_routes_list)
uncompleted_routes_df = pd.DataFrame(uncompleted_routes_list)

print(len(completed_routes_df), len(uncompleted_routes_df))
uncompleted_routes_df

8612 9950


,planned_route_craft,actual_route_location,driver_id,day_of_week,last_two_weeks_count,location_type_id,stop_arrived_at,stop_earliest,stop_latest,current_lat,current_lng,difference_distance,distance_route,distance_actual_route,location_is_depot,date,planned_route_location,actual_route_unique
0,"[1167, 2231, 2406, 2349, 2298, 2588, 2493]","[1167, 2231, 2349, 2406, 2298, 2493, 2588]",1186,Tuesday,9,"[0, 1, 1, 1, 1, 1, 1]","[2024-01-15 22:32:42.615543+00, 2024-01-15 23:...","[2024-01-15 19:00:00+00, 2024-01-15 21:00:00+0...","[2024-01-16 01:00:00+00, 2024-01-16 04:00:00+0...","[55.64641920439919, 55.7498333, 55.80148, 55.7...","[12.15372974969827, 12.5870785, 12.525814, 12....",-0.027603,"[18.380690356980487, 4.298068156860068, 1.5617...","[18.380690356980487, 3.164379243258826, 1.5617...","[1, 0, 0, 0, 0, 0, 0]",2024-01-15,"[132992, 128251, 128480, 129701, 130263, 13014...","[132992, 128251, 129701, 128480, 130263, 14652..."
1,"[1167, 1168, 1495, 1658, 1070, 818, 965, 936, ...","[1167, 1070, 936, 822, 963, 965, 818, 1658, 14...",1218,Tuesday,11,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[2024-01-15 22:37:07.860851+00, 2024-01-16 00:...","[2024-01-15 19:00:00+00, 2024-01-15 21:00:00+0...","[2024-01-16 01:00:00+00, 2024-01-16 04:00:00+0...","[55.64641920439919, 55.646535, 55.672309, 55.6...","[12.15372974969827, 12.539958, 12.588959, 12.6...",0.435438,"[15.110089331615377, 2.617611669361755, 0.8878...","[19.4862183744275, 11.162649116186252, 5.20670...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",2024-01-15,"[132992, 128114, 129560, 128017, 128413, 12798...","[132992, 128413, 128394, 129091, 128617, 12901..."
2,"[1167, 1101, 1079, 1282, 1123, 1098, 834, 1262...","[1167, 1098, 1315, 1282, 1262, 834, 1123, 1079...",1192,Tuesday,11,"[0, 1, 1, 1, 1, 1, 1, 1, 1]","[2024-01-15 23:06:33.822316+00, 2024-01-16 01:...","[2024-01-15 19:00:00+00, 2024-01-15 21:00:00+0...","[2024-01-16 01:00:00+00, 2024-01-16 04:00:00+0...","[55.64641920439919, 55.640375, 55.639120600000...","[12.15372974969827, 12.091312, 12.0627714, 12....",0.162129,"[2.4776512575967895, 1.1201296974431052, 1.598...","[2.5909389807670826, 2.6002283089649945, 2.104...","[1, 0, 0, 0, 0, 0, 0, 0, 0]",2024-01-15,"[132992, 127628, 128567, 128593, 127977, 12878...","[132992, 128788, 128647, 128593, 128612, 12860..."
3,"[1167, 1951, 1951, 1777, 1796, 1811, 1949, 196...","[1167, 1999, 1929, 1951, 1951, 1777, 1796, 181...",1198,Tuesday,8,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[2024-01-15 22:08:46.207788+00, 2024-01-16 01:...","[2024-01-15 19:00:00+00, 2024-01-15 23:00:00+0...","[2024-01-16 01:00:00+00, 2024-01-16 08:30:00+0...","[55.64641920439919, 55.712176, 55.712176, 55.6...","[12.15372974969827, 12.541758, 12.541758, 12.5...",0.114027,"[15.835300652256178, 0.0, 1.5016735562871621, ...","[16.720642190707288, 0.31770677054599067, 0.84...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",2024-01-15,"[132992, 128181, 128177, 127738, 128193, 13045...","[132992, 130491, 147908, 128177, 128181, 12773..."
4,"[1167, 1420, 1519, 1687, 1528, 1367, 1174, 1174]","[1167, 1174, 1174, 1367, 1528, 1687, 1519, 1420]",1210,Tuesday,10,"[0, 1, 1, 1, 1, 1, 1, 1]","[2024-01-15 21:51:16.697854+00, 2024-01-16 01:...","[2024-01-15 19:00:00+00, 2024-01-15 21:00:00+0...","[2024-01-16 01:00:00+00, 2024-01-16 04:00:00+0...","[55.64641920439919, 55.66807900000001, 55.6733...","[12.15372974969827, 12.562603, 12.5660271, 12....",-0.231637,"[16.061664613017072, 0.3894836165795559, 0.958...","[10.302891542998038, 0.0, 5.823064259213148, 0...","[1, 0, 0, 0, 0, 0, 0, 0]",2024-01-15,"[132992, 129648, 127603, 130480, 127950, 12810...","[132992, 146399, 146401, 128107, 127950, 13048..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9945,"[3162, 3243, 3295, 3295, 3280, 3249, 3281, 329...","[3162, 3243, 3295, 3295, 3281, 3296, 3273, 327...",1520,Tuesday,2,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2024-08-06 01:57:14.727094+00, 2024-08-06 02:...","[2024-08-06 00:00:00+00, 2024-08-06 01:00:00+0...",

In [20]:
drivers_dic = {}
k = 1
for driver in completed_routes_df['driver_id']:
    if driver not in drivers_dic:
        drivers_dic[driver] = k
        k += 1
print('Total number of drivers', len(drivers_dic))
total_drivers = len(drivers_dic)
encoding_drivers = []
for driver in completed_routes_df['driver_id']:
    encoding_drivers.append(drivers_dic[driver])
#
completed_routes_df['driver_id_sorted'] = encoding_drivers
# final_routes = pd.concat([final_routes, pd.get_dummies(final_routes['driver_id_sorted'], prefix='encoding_drivers')], axis=1)
completed_routes_df

Total number of drivers 379


,planned_route_craft,actual_route_location,driver_id,day_of_week,last_two_weeks_count,location_type_id,stop_arrived_at,stop_earliest,stop_latest,current_lat,current_lng,difference_distance,distance_route,distance_actual_route,location_is_depot,date,planned_route_location,actual_route_unique,driver_id_sorted
0,"[1167, 1405, 1560, 1413, 1391, 1391, 1434, 1269]","[1167, 1405, 1560, 1413, 1391, 1391, 1434, 1269]",1255,Tuesday,8,"[0, 1, 1, 1, 1, 1, 1, 1]","[2024-01-15 22:11:49.528537+00, 2024-01-15 23:...","[2024-01-15 19:00:00+00, 2024-01-15 23:00:00+0...","[2024-01-16 01:00:00+00, 2024-01-16 03:00:00+0...","[55.64641920439919, 55.6669147, 55.6757322, 55...","[12.15372974969827, 12.5262801, 12.5619711, 12...",0.000000,"[14.640018023192697, 1.5229312527257717, 0.606...","[14.640018023192697, 1.5229312527257717, 0.606...","[1, 0, 0, 0, 0, 0, 0, 0]",2024-01-15,"[132992, 128289, 128048, 128448, 128037, 12845...","[132992, 128289, 128048, 128448, 128454, 12803...",1
1,"[1167, 1352, 1312, 1356, 1508, 1508, 1621, 1401]","[1167, 1312, 1356, 1352, 1508, 1508, 1621, 1401]",1195,Monday,11,"[0, 1, 1, 1, 1, 1, 1, 1]","[2024-01-15 22:58:58.038139+00, 2024-01-15 23:...","[2024-01-15 19:00:00+00, 2024-01-15 21:00:00+0...","[2024-01-16 01:00:00+00, 2024-01-16 04:00:00+0...","[55.64641920439919, 55.662018, 55.658103, 55.6...","[12.15372974969827, 12.360745, 12.368506, 12.3...",0.006369,"[8.16888775040224, 0.4067943329231743, 0.35828...","[8.440075587409163, 0.35828887642036544, 0.110...","[1, 0, 0, 0, 0, 0, 0, 0]",2024-01-15,"[132992, 130073, 146768, 130075, 128234, 12823...","[132992, 146768, 130075, 130073, 128234, 12823...",2
2,"[1167, 1405, 1479, 1560, 1413, 1391, 1391, 143...","[1167, 1405, 1479, 1560, 1413, 1391, 1391, 143...",1255,Wednesday,7,"[0, 1, 1, 1, 1, 1, 1, 1, 1]","[2024-01-16 22:24:33.871644+00, 2024-01-16 23:...","[2024-01-16 19:00:00+00, 2024-01-16 23:00:00+0...","[2024-01-17 01:00:00+00, 2024-01-17 03:00:00+0...","[55.64641920439919, 55.6669147, 55.671473, 55....","[12.15372974969827, 12.5262801, 12.561174, 12....",0.000000,"[14.640018023192697, 1.400306417188586, 0.2962...","[14.640018023192697, 1.400306417188586, 0.2962...","[1, 0, 0, 0, 0, 0, 0, 0, 0]",2024-01-16,"[132992, 128289, 127766, 128048, 128448, 12803...","[132992, 128289, 127766, 128048, 128448, 12845...",1
3,"[1167, 1194, 947, 1797, 1167]","[1167, 1194, 947, 1797, 1167]",1190,Tuesday,2,"[0, 1, 1, 0, 1]","[2024-01-23 10:49:20.166468+00, 2024-01-23 10:...","[2024-01-16 00:00:00+00, 2024-01-16 05:00:00+0...","[2024-01-16 22:00:00+00, 2024-01-16 11:00:00+0...","[55.64641920439919, 55.6487125, 55.6163424, 55...","[12.15372974969827, 12.1307559, 12.3275461, 12...",0.000000,"[0.9126584187089052, 8.020566881548852, 6.6327...","[0.9126584187089052, 8.020566881548852, 6.6327...","[1, 0, 0, 0, 1]",2024-01-23,"[132992, 127580, 127584, 164074, 132992]","[132992, 127580, 127584, 164074, 132992]",3
4,"[1167, 696, 662, 615, 614, 628, 628, 634, 633,...","[1167, 696, 662, 615, 614, 628, 628, 634, 633,...",1178,Tuesday,8,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2024-01-23 10:49:20.775856+00, 2024-01-23 10:...","[2024-01-16 00:00:00+00, 2024-01-15 23:00:00+0...","[2024-01-16 22:00:00+00, 2024-01-16 07:30:00+0...","[55.64641920439919, 55.4520374, 55.4421082, 55...","[12.15372974969827, 12.1856949, 12.1677321, 12...",0.000000,"[13.505545464972036, 0.9852343060237925, 2.970...","[13.505545464972036, 0.9852343060237925, 2.970...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2024-01-23,"[132992, 129376, 129449, 129405, 129868, 12931...","[132992, 129376, 129449, 129405, 129868, 12931...",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8607,"[10076, 7957, 10076, 9131, 6485, 6362, 6086, 5...","[10076, 7957, 10076, 9131, 6485, 6362, 6086, 5...",274,Wednesday,1,"[0, 1, 0, 1, 1, 1, 1, 1, 1]","[2024-08-07 15:24:29.958093+00, 2024-08-07 15:...","[2024-08-07 01:00:00+00, 2024-08-07 05:00:00+0...","[2024-08-07 14:00:00+00, 2024-08-07 11:00:00+0..

In [21]:
uncompleted_routes_df

,planned_route_craft,actual_route_location,driver_id,day_of_week,last_two_weeks_count,location_type_id,stop_arrived_at,stop_earliest,stop_latest,current_lat,current_lng,difference_distance,distance_route,distance_actual_route,location_is_depot,date,planned_route_location,actual_route_unique
0,"[1167, 2231, 2406, 2349, 2298, 2588, 2493]","[1167, 2231, 2349, 2406, 2298, 2493, 2588]",1186,Tuesday,9,"[0, 1, 1, 1, 1, 1, 1]","[2024-01-15 22:32:42.615543+00, 2024-01-15 23:...","[2024-01-15 19:00:00+00, 2024-01-15 21:00:00+0...","[2024-01-16 01:00:00+00, 2024-01-16 04:00:00+0...","[55.64641920439919, 55.7498333, 55.80148, 55.7...","[12.15372974969827, 12.5870785, 12.525814, 12....",-0.027603,"[18.380690356980487, 4.298068156860068, 1.5617...","[18.380690356980487, 3.164379243258826, 1.5617...","[1, 0, 0, 0, 0, 0, 0]",2024-01-15,"[132992, 128251, 128480, 129701, 130263, 13014...","[132992, 128251, 129701, 128480, 130263, 14652..."
1,"[1167, 1168, 1495, 1658, 1070, 818, 965, 936, ...","[1167, 1070, 936, 822, 963, 965, 818, 1658, 14...",1218,Tuesday,11,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[2024-01-15 22:37:07.860851+00, 2024-01-16 00:...","[2024-01-15 19:00:00+00, 2024-01-15 21:00:00+0...","[2024-01-16 01:00:00+00, 2024-01-16 04:00:00+0...","[55.64641920439919, 55.646535, 55.672309, 55.6...","[12.15372974969827, 12.539958, 12.588959, 12.6...",0.435438,"[15.110089331615377, 2.617611669361755, 0.8878...","[19.4862183744275, 11.162649116186252, 5.20670...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",2024-01-15,"[132992, 128114, 129560, 128017, 128413, 12798...","[132992, 128413, 128394, 129091, 128617, 12901..."
2,"[1167, 1101, 1079, 1282, 1123, 1098, 834, 1262...","[1167, 1098, 1315, 1282, 1262, 834, 1123, 1079...",1192,Tuesday,11,"[0, 1, 1, 1, 1, 1, 1, 1, 1]","[2024-01-15 23:06:33.822316+00, 2024-01-16 01:...","[2024-01-15 19:00:00+00, 2024-01-15 21:00:00+0...","[2024-01-16 01:00:00+00, 2024-01-16 04:00:00+0...","[55.64641920439919, 55.640375, 55.639120600000...","[12.15372974969827, 12.091312, 12.0627714, 12....",0.162129,"[2.4776512575967895, 1.1201296974431052, 1.598...","[2.5909389807670826, 2.6002283089649945, 2.104...","[1, 0, 0, 0, 0, 0, 0, 0, 0]",2024-01-15,"[132992, 127628, 128567, 128593, 127977, 12878...","[132992, 128788, 128647, 128593, 128612, 12860..."
3,"[1167, 1951, 1951, 1777, 1796, 1811, 1949, 196...","[1167, 1999, 1929, 1951, 1951, 1777, 1796, 181...",1198,Tuesday,8,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[2024-01-15 22:08:46.207788+00, 2024-01-16 01:...","[2024-01-15 19:00:00+00, 2024-01-15 23:00:00+0...","[2024-01-16 01:00:00+00, 2024-01-16 08:30:00+0...","[55.64641920439919, 55.712176, 55.712176, 55.6...","[12.15372974969827, 12.541758, 12.541758, 12.5...",0.114027,"[15.835300652256178, 0.0, 1.5016735562871621, ...","[16.720642190707288, 0.31770677054599067, 0.84...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",2024-01-15,"[132992, 128181, 128177, 127738, 128193, 13045...","[132992, 130491, 147908, 128177, 128181, 12773..."
4,"[1167, 1420, 1519, 1687, 1528, 1367, 1174, 1174]","[1167, 1174, 1174, 1367, 1528, 1687, 1519, 1420]",1210,Tuesday,10,"[0, 1, 1, 1, 1, 1, 1, 1]","[2024-01-15 21:51:16.697854+00, 2024-01-16 01:...","[2024-01-15 19:00:00+00, 2024-01-15 21:00:00+0...","[2024-01-16 01:00:00+00, 2024-01-16 04:00:00+0...","[55.64641920439919, 55.66807900000001, 55.6733...","[12.15372974969827, 12.562603, 12.5660271, 12....",-0.231637,"[16.061664613017072, 0.3894836165795559, 0.958...","[10.302891542998038, 0.0, 5.823064259213148, 0...","[1, 0, 0, 0, 0, 0, 0, 0]",2024-01-15,"[132992, 129648, 127603, 130480, 127950, 12810...","[132992, 146399, 146401, 128107, 127950, 13048..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9945,"[3162, 3243, 3295, 3295, 3280, 3249, 3281, 329...","[3162, 3243, 3295, 3295, 3281, 3296, 3273, 327...",1520,Tuesday,2,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2024-08-06 01:57:14.727094+00, 2024-08-06 02:...","[2024-08-06 00:00:00+00, 2024-08-06 01:00:00+0...",

In [22]:
sunday_count = completed_routes_df[completed_routes_df["day_of_week"] == "Sunday"].shape[0]
print(f"Number of rows on Sunday: {sunday_count}")

Number of rows on Sunday: 27


In [23]:

# completed_routes_list_unique = [array for i, array in enumerate(completed_routes_list) if array not in completed_routes_list[:i]]
# uncompleted_routes_list_unique = [array for i, array in enumerate(uncompleted_routes_list) if array not in uncompleted_routes_list[:i]]
# len(completed_routes_list_unique), len (uncompleted_routes_list_unique)
# len(completed_routes_df), len(uncompleted_routes_df)

In [38]:
final_routes = pd.DataFrame({
    'routes': completed_routes_df['planned_route_craft'].tolist() + uncompleted_routes_df['planned_route_craft'].tolist(),
    'actual_routes': completed_routes_df['actual_route_location'].tolist() + uncompleted_routes_df['actual_route_location'].tolist(),
    'driver_id': completed_routes_df['driver_id'].tolist() +  uncompleted_routes_df['driver_id'].tolist(),
    'distance_route': completed_routes_df['distance_route'].tolist() + uncompleted_routes_df['distance_route'].tolist(),
    'distance_actual_route': completed_routes_df['distance_actual_route'].tolist() + uncompleted_routes_df['distance_actual_route'].tolist(),
    'last_two_weeks_count': completed_routes_df['last_two_weeks_count'].tolist() + uncompleted_routes_df['last_two_weeks_count'].tolist(),
    'current_lat': completed_routes_df['current_lat'].tolist() + uncompleted_routes_df['current_lat'].tolist(),
    'current_lng': completed_routes_df['current_lng'].tolist() + uncompleted_routes_df['current_lng'].tolist(),
    'day_of_week': completed_routes_df['day_of_week'].tolist() + uncompleted_routes_df['day_of_week'].tolist(),
    'date': completed_routes_df['date'].tolist() + uncompleted_routes_df['date'].tolist(),
    'location_is_depot': completed_routes_df['location_is_depot'].tolist() + uncompleted_routes_df['location_is_depot'].tolist(),
    'location_type_id': completed_routes_df['location_type_id'].tolist() + uncompleted_routes_df['location_type_id'].tolist(),
    'arriving_time': completed_routes_df['stop_arrived_at'].tolist() + uncompleted_routes_df['stop_arrived_at'].tolist(),
    'planned_route_location': completed_routes_df['planned_route_location'].tolist() + uncompleted_routes_df['planned_route_location'].tolist(),
    'actual_route_unique': completed_routes_df['actual_route_unique'].tolist() + uncompleted_routes_df['actual_route_unique'].tolist(),
    'stop_earliest': completed_routes_df['stop_earliest'].tolist() + uncompleted_routes_df['stop_earliest'].tolist(),
    'stop_latest': completed_routes_df['stop_latest'].tolist() + uncompleted_routes_df['stop_latest'].tolist(),
    'label': [0] * len(completed_routes_df)  + [1] * len(uncompleted_routes_df)
})
final_routes['len'] = final_routes['routes'].apply(lambda x: len(x))
final_routes

# [0] * len(artificial_planned_routes)
# artificial_planned_routes['distance_route'].tolist()
# artificial_planned_routes['driver_id'].tolist()
# artificial_planned_routes['common_subsequence'].tolist()

,routes,actual_routes,driver_id,distance_route,distance_actual_route,last_two_weeks_count,current_lat,current_lng,day_of_week,date,location_is_depot,location_type_id,arriving_time,planned_route_location,actual_route_unique,stop_earliest,stop_latest,label,len
0,"[1167, 1405, 1560, 1413, 1391, 1391, 1434, 1269]","[1167, 1405, 1560, 1413, 1391, 1391, 1434, 1269]",1255,"[14.640018023192697, 1.5229312527257717, 0.606...","[14.640018023192697, 1.5229312527257717, 0.606...",8,"[55.64641920439919, 55.6669147, 55.6757322, 55...","[12.15372974969827, 12.5262801, 12.5619711, 12...",Tuesday,2024-01-15,"[1, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 1, 1, 1, 1, 1, 1]","[2024-01-15 22:11:49.528537+00, 2024-01-15 23:...","[132992, 128289, 128048, 128448, 128037, 12845...","[132992, 128289, 128048, 128448, 128454, 12803...","[2024-01-15 19:00:00+00, 2024-01-15 23:00:00+0...","[2024-01-16 01:00:00+00, 2024-01-16 03:00:00+0...",0,8
1,"[1167, 1352, 1312, 1356, 1508, 1508, 1621, 1401]","[1167, 1312, 1356, 1352, 1508, 1508, 1621, 1401]",1195,"[8.16888775040224, 0.4067943329231743, 0.35828...","[8.440075587409163, 0.35828887642036544, 0.110...",11,"[55.64641920439919, 55.662018, 55.658103, 55.6...","[12.15372974969827, 12.360745, 12.368506, 12.3...",Monday,2024-01-15,"[1, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 1, 1, 1, 1, 1, 1]","[2024-01-15 22:58:58.038139+00, 2024-01-15 23:...","[132992, 130073, 146768, 130075, 128234, 12823...","[132992, 146768, 130075, 130073, 128234, 12823...","[2024-01-15 19:00:00+00, 2024-01-15 21:00:00+0...","[2024-01-16 01:00:00+00, 2024-01-16 04:00:00+0...",0,8
2,"[1167, 1405, 1479, 1560, 1413, 1391, 1391, 143...","[1167, 1405, 1479, 1560, 1413, 1391, 1391, 143...",1255,"[14.640018023192697, 1.400306417188586, 0.2962...","[14.640018023192697, 1.400306417188586, 0.2962...",7,"[55.64641920439919, 55.6669147, 55.671473, 55....","[12.15372974969827, 12.5262801, 12.561174, 12....",Wednesday,2024-01-16,"[1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 1, 1, 1, 1, 1, 1, 1]","[2024-01-16 22:24:33.871644+00, 2024-01-16 23:...","[132992, 128289, 127766, 128048, 128448, 12803...","[132992, 128289, 127766, 128048, 128448, 12845...","[2024-01-16 19:00:00+00, 2024-01-16 23:00:00+0...","[2024-01-17 01:00:00+00, 2024-01-17 03:00:00+0...",0,9
3,"[1167, 1194, 947, 1797, 1167]","[1167, 1194, 947, 1797, 1167]",1190,"[0.9126584187089052, 8.020566881548852, 6.6327...","[0.9126584187089052, 8.020566881548852, 6.6327...",2,"[55.64641920439919, 55.6487125, 55.6163424, 55...","[12.15372974969827, 12.1307559, 12.3275461, 12...",Tuesday,2024-01-23,"[1, 0, 0, 0, 1]","[0, 1, 1, 0, 1]","[2024-01-23 10:49:20.166468+00, 2024-01-23 10:...","[132992, 127580, 127584, 164074, 132992]","[132992, 127580, 127584, 164074, 132992]","[2024-01-16 00:00:00+00, 2024-01-16 05:00:00+0...","[2024-01-16 22:00:00+00, 2024-01-16 11:00:00+0...",0,5
4,"[1167, 696, 662, 615, 614, 628, 628, 634, 633,...","[1167, 696, 662, 615, 614, 628, 628, 634, 633,...",1178,"[13.505545464972036, 0.9852343060237925, 2.970...","[13.505545464972036, 0.9852343060237925, 2.970...",8,"[55.64641920439919, 55.4520374, 55.4421082, 55...","[12.15372974969827, 12.1856949, 12.1677321, 12...",Tuesday,2024-01-23,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2024-01-23 10:49:20.775856+00, 2024-01-23 10:...","[132992, 129376, 129449, 129405, 129868, 12931...","[132992, 129376, 129449, 129405, 129868, 12931...","[2024-01-16 00:00:00+00, 2024-01-15 23:00:00+0...","[2024-01-16 22:00:00+00, 2024-01-16 07:30:00+0...",0,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18557,"[3162, 3243, 3295, 3295, 3280, 3249, 3281, 329...","[3162, 3243, 3295, 3295, 3281, 3296, 3273, 327...",1520,"[4.097697796278882, 2.827734737537246, 0.0, 0....","[4.097697796278882, 2.827734737537246, 0.0, 2....",2,"[56.0667627, 56.1005391, 56.117045, 56.117045,...","[9.986944, 10.0739552, 10.1408688, 10.1408688,...",Tuesday,2024-08-05,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, .

In [39]:
def determine_country(lat, lng):
    if 57.5 <= lat <= 71.2 and 4.0 <= lng <= 31.0:
        return 0  # Norway
    elif 54.5 <= lat <= 57.8 and 8.0 <= lng <= 15.0:
        return 1  # Denmark
    else:
        return 1
final_routes['country_flag'] = final_routes.apply(lambda row: determine_country(row['current_lat'][0], row['current_lng'][0]), axis=1)

max([len(i) for i in final_routes['routes']])

36

In [40]:
#create dictionary and encode to smaller unique numbers
locations_dic = {}
location_count = {}
k = 1
for row in final_routes['routes']:
   for location in row:
       if location not in locations_dic:
           locations_dic[location] = k
           k += 1
k = 1
print(len(locations_dic))

for row in final_routes['routes']:
   for location in row:
       if location not in location_count:
            location_count[location] = 1
       else:
            location_count[location] += 1


10704


In [41]:
drivers_dic = {}
k = 1
for driver in final_routes['driver_id']:
    if driver not in drivers_dic:
        drivers_dic[driver] = k
        k += 1
print('Total number of drivers', len(drivers_dic))
total_drivers = len(drivers_dic)
encoding_drivers = []
for driver in final_routes['driver_id']:
    encoding_drivers.append(drivers_dic[driver])
#
final_routes['driver_id_sorted'] = encoding_drivers
final_routes['day_of_week_encoded'] = pd.Categorical(final_routes['day_of_week']).codes+1

def encode_day_of_week(row):
    a = np.zeros((36, 7), dtype=int)
    route_len = row['len']
    day_encoded = row['day_of_week_encoded']
    for i in range(36 - route_len, len(a)):
        a[i][day_encoded-1] = 1
    return a

def encode_day_of_week_simple(row):
    a = np.zeros(7, dtype=int)
    day_encoded = row['day_of_week_encoded']
    a[day_encoded - 1] = 1
    return a

final_routes['experience_feature'] = final_routes.apply(lambda x: x['len'] * [x['last_two_weeks_count']], axis = 1)
final_routes['len_feature'] = final_routes.apply(lambda x: x['len'] * [x['len']], axis = 1)
final_routes['driver_id_feature'] = final_routes.apply(lambda x: x['len'] * [x['driver_id_sorted']], axis = 1)
final_routes['country_flag_feature'] = final_routes.apply(lambda x: x['len'] * [x['country_flag']], axis = 1)
final_routes['day_of_week_feature'] = final_routes.apply(encode_day_of_week, axis = 1)
final_routes['day_of_week_encoded_ext'] = final_routes.apply(encode_day_of_week_simple, axis = 1)

# encoding_routes = []
# for row in final_routes['routes']:
#     encoding_route = []
#     for location in row:
#         encoding_route.append(locations_dic[location])
#     encoding_routes.append(encoding_route)
# final_routes['routes'] = encoding_routes

print('correlation', final_routes['len'].corr(final_routes['label']))
print('correlation', max(final_routes['len']))
# final_routes = pd.concat([final_routes, pd.get_dummies(final_routes['driver_id_sorted'], prefix='encoding_drivers')], axis=1)

Total number of drivers 395
correlation 0.3069126041002025
correlation 36


In [42]:
final_routes['first_arrival'] = final_routes['arriving_time'].apply(lambda x: x[0])

# Convert to datetime and sort
final_routes['first_arrival'] = pd.to_datetime(final_routes['first_arrival'])
final_routes = final_routes.sort_values('first_arrival').reset_index(drop=True)

# Drop the temporary column
final_routes = final_routes.drop('first_arrival', axis=1)
final_routes

,routes,actual_routes,driver_id,distance_route,distance_actual_route,last_two_weeks_count,current_lat,current_lng,day_of_week,date,...,len,country_flag,driver_id_sorted,day_of_week_encoded,experience_feature,len_feature,driver_id_feature,country_flag_feature,day_of_week_feature,day_of_week_encoded_ext
0,"[1167, 574, 1167, 523, 473, 462, 444, 400, 190...","[1167, 1167, 574, 523, 473, 462, 444, 400, 190...",1184,"[23.479040480510506, 23.479040480510506, 26.97...","[0.0, 23.479040480510506, 8.146314511821483, 3...",8,"[55.64641920439919, 55.32612200000001, 55.6464...","[12.15372974969827, 11.9560479, 12.15372974969...",Monday,2024-01-15,...,16,1,7,2,"[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]","[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 1...","[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[0, 1, 0, 0, 0, 0, 0]"
1,"[1167, 1204, 1042, 1039, 1033, 1033, 1030, 127...","[1167, 1204, 1042, 1039, 1030, 1275, 962, 1033...",1486,"[1.141374091100897, 2.8522152887994903, 1.5389...","[1.141374091100897, 2.8522152887994903, 1.5389...",2,"[55.64641920439919, 55.649076, 55.63469, 55.63...","[12.15372974969827, 12.124935, 12.05662, 12.01...",Monday,2024-01-15,...,13,1,146,2,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]","[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...","[146, 146, 146, 146, 146, 146, 146, 146, 146, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[0, 1, 0, 0, 0, 0, 0]"
2,"[7166, 7166, 8045, 8718, 8457, 8093, 8310, 840...","[7166, 7166, 8045, 8718, 8457, 8093, 8310, 840...",1432,"[0.0, 16.752201025499367, 1.501855696871057, 0...","[0.0, 16.752201025499367, 1.501855696871057, 0...",8,"[59.805664, 59.805664, 59.910491, 59.922304900...","[10.290383, 10.290383, 10.723998, 10.6877597, ...",Monday,2024-01-15,...,13,0,23,2,"[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]","[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...","[23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[0, 1, 0, 0, 0, 0, 0]"
3,"[7166, 10203, 10214, 10214, 10623, 10692, 1066...","[7166, 10203, 10214, 10214, 10623, 10692, 1066...",723,"[31.65996164052381, 0.43893999280403884, 0.0, ...","[31.65996164052381, 0.43893999280403884, 0.0, ...",4,"[59.805664, 60.066484, 60.072593, 60.072593, 6...","[10.290383, 11.039009, 11.035613, 11.035613, 1...",Monday,2024-01-15,...,8,0,113,2,"[4, 4, 4, 4, 4, 4, 4, 4]","[8, 8, 8, 8, 8, 8, 8, 8]","[113, 113, 113, 113, 113, 113, 113, 113]","[0, 0, 0, 0, 0, 0, 0, 0]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[0, 1, 0, 0, 0, 0, 0]"
4,"[1167, 2331, 2299, 2294, 2383, 2385, 2397, 240...","[1167, 2383, 2299, 2294, 2381, 2390, 2384, 235...",1324,"[8.678113362175365, 0.7124995743679728, 0.2457...","[10.893805973040553, 3.7200180786093084, 0.245...",4,"[55.64641920439919, 55.77168, 55.766721, 55.76...","[12.15372974969827, 12.165606, 12.181618, 12.1...",Monday,2024-01-15,...,13,1,27,2,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]","[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...","[27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[0, 1, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18557,"[1167, 35, 38, 41, 45, 91, 110, 134, 121, 125,...","[1167, 35, 38, 41, 45, 91, 110, 134, 121, 125,...",1243,"[68.89851202059795, 1.2509793817380201, 4.9521...","[68.89851202059795, 1.2509793817380201, 4.9521...",1,"[55.64641920439919, 54.696715, 54.6986631, 54....","[12.15372974969827, 11.629326, 11.598282, 11.4...",Thursday,2024-08-07,...,11,1,17,5,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11]","[17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],

In [43]:
# np.random.seed(42)
# final_routes['date'] = pd.to_datetime(final_routes['date'])
# final_routes['tie_breaker'] = range(len(final_routes))
# final_routes = final_routes.sort_values(by=['date', 'tie_breaker'], ascending=[True, True])
# final_routes = final_routes.reset_index(drop=True)
# final_routes = final_routes.drop('tie_breaker', axis=1)
# final_routes

In [44]:
final_routes

,routes,actual_routes,driver_id,distance_route,distance_actual_route,last_two_weeks_count,current_lat,current_lng,day_of_week,date,...,len,country_flag,driver_id_sorted,day_of_week_encoded,experience_feature,len_feature,driver_id_feature,country_flag_feature,day_of_week_feature,day_of_week_encoded_ext
0,"[1167, 574, 1167, 523, 473, 462, 444, 400, 190...","[1167, 1167, 574, 523, 473, 462, 444, 400, 190...",1184,"[23.479040480510506, 23.479040480510506, 26.97...","[0.0, 23.479040480510506, 8.146314511821483, 3...",8,"[55.64641920439919, 55.32612200000001, 55.6464...","[12.15372974969827, 11.9560479, 12.15372974969...",Monday,2024-01-15,...,16,1,7,2,"[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]","[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 1...","[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[0, 1, 0, 0, 0, 0, 0]"
1,"[1167, 1204, 1042, 1039, 1033, 1033, 1030, 127...","[1167, 1204, 1042, 1039, 1030, 1275, 962, 1033...",1486,"[1.141374091100897, 2.8522152887994903, 1.5389...","[1.141374091100897, 2.8522152887994903, 1.5389...",2,"[55.64641920439919, 55.649076, 55.63469, 55.63...","[12.15372974969827, 12.124935, 12.05662, 12.01...",Monday,2024-01-15,...,13,1,146,2,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]","[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...","[146, 146, 146, 146, 146, 146, 146, 146, 146, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[0, 1, 0, 0, 0, 0, 0]"
2,"[7166, 7166, 8045, 8718, 8457, 8093, 8310, 840...","[7166, 7166, 8045, 8718, 8457, 8093, 8310, 840...",1432,"[0.0, 16.752201025499367, 1.501855696871057, 0...","[0.0, 16.752201025499367, 1.501855696871057, 0...",8,"[59.805664, 59.805664, 59.910491, 59.922304900...","[10.290383, 10.290383, 10.723998, 10.6877597, ...",Monday,2024-01-15,...,13,0,23,2,"[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]","[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...","[23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[0, 1, 0, 0, 0, 0, 0]"
3,"[7166, 10203, 10214, 10214, 10623, 10692, 1066...","[7166, 10203, 10214, 10214, 10623, 10692, 1066...",723,"[31.65996164052381, 0.43893999280403884, 0.0, ...","[31.65996164052381, 0.43893999280403884, 0.0, ...",4,"[59.805664, 60.066484, 60.072593, 60.072593, 6...","[10.290383, 11.039009, 11.035613, 11.035613, 1...",Monday,2024-01-15,...,8,0,113,2,"[4, 4, 4, 4, 4, 4, 4, 4]","[8, 8, 8, 8, 8, 8, 8, 8]","[113, 113, 113, 113, 113, 113, 113, 113]","[0, 0, 0, 0, 0, 0, 0, 0]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[0, 1, 0, 0, 0, 0, 0]"
4,"[1167, 2331, 2299, 2294, 2383, 2385, 2397, 240...","[1167, 2383, 2299, 2294, 2381, 2390, 2384, 235...",1324,"[8.678113362175365, 0.7124995743679728, 0.2457...","[10.893805973040553, 3.7200180786093084, 0.245...",4,"[55.64641920439919, 55.77168, 55.766721, 55.76...","[12.15372974969827, 12.165606, 12.181618, 12.1...",Monday,2024-01-15,...,13,1,27,2,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]","[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...","[27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[0, 1, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18557,"[1167, 35, 38, 41, 45, 91, 110, 134, 121, 125,...","[1167, 35, 38, 41, 45, 91, 110, 134, 121, 125,...",1243,"[68.89851202059795, 1.2509793817380201, 4.9521...","[68.89851202059795, 1.2509793817380201, 4.9521...",1,"[55.64641920439919, 54.696715, 54.6986631, 54....","[12.15372974969827, 11.629326, 11.598282, 11.4...",Thursday,2024-08-07,...,11,1,17,5,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11]","[17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],

In [37]:
from collections import defaultdict

def analyze_routes(df):
    rows = []

    for idx, row in df.iterrows():
        route_id = idx
        driver_id = row['driver_id_sorted']
        planned_real = row['planned_route_location']
        actual_real = row['actual_route_unique']
        arriving_times = row['arriving_time']
        planned_distances = row['distance_route']
        actual_distances = row['distance_actual_route']
        country_flag = row['country_flag']
        day_of_week = row['day_of_week']
        date = row['date']
        experience = row['last_two_weeks_count']
        is_depot = row['location_is_depot']
        craft_id = row['routes']

        #times
        arr_times = [pd.to_datetime(ts) for ts in arriving_times]
        earliest_times = [pd.to_datetime(ts) for ts in row['earliest_times']]
        latest_times = [pd.to_datetime(ts) for ts in row['latest_times']]
        all_times = [arr_times, earliest_times, latest_times]
        arr_durations = [(t - arr_times[0]).total_seconds()  for t in arr_times]

        #indexes
        map = defaultdict(lambda: [])
        for i in range(len(planned_real)):
            map[planned_real[i]].append(i)

        map_index = defaultdict(lambda: 0)

        for (idx, stop) in enumerate(actual_real):
            planned_index = idx
            planned_distance = 0 if planned_index == 0 else planned_distances[planned_index - 1]
            if stop in map:
                actual_index = map[stop][map_index[stop]]
                map_index[stop] += 1
            else:
                actual_index = planned_real.index(stop)
            actual_distance = 0 if actual_index == 0 else actual_distances[actual_index - 1]

            rows.append({
                'Route ID': route_id,
                'Driver ID': driver_id,
                'Country': country_flag,
                'Day of Week': day_of_week,
                'Stop ID': stop-1,
                'Location ID': craft_id[idx],
                'IndexP': planned_index,
                'IndexA': actual_index,
                'Visit Time': visit_time,
                'DistanceP': planned_distance,
                'DistanceA': actual_distance
            })

    return pd.DataFrame(rows)

newdf = analyze_routes(final_routes)
newdf.to_csv('routes_driver_analysis.csv', index=False)

['2024-01-15 02:30:31.162054+00', '2024-01-15 03:37:34.655985+00', '2024-01-15 02:33:18.999068+00', '2024-01-15 04:21:40.479367+00', '2024-01-15 04:42:08.083062+00', '2024-01-15 05:27:38.858569+00', '2024-01-15 05:41:08.376625+00', '2024-01-15 06:17:27.001502+00', '2024-01-15 07:06:52.536473+00', '2024-01-15 07:59:54.966326+00', '2024-01-15 07:51:59.351042+00', '2024-01-15 09:22:28.131422+00', '2024-01-15 09:30:07.68767+00', '2024-01-15 08:45:53.218983+00', '2024-01-15 08:37:35.625759+00', '2024-01-15 08:19:42.279029+00'] [0.0, 4023.493931, 167.837014, 6669.317313, 7896.921008, 10627.696515, 11437.214571, 13615.839448, 16581.374419, 19763.804272, 19288.188988, 24716.969368, 25176.525616, 22522.056929, 22024.463705, 20951.116975]


TypeError: exceptions must derive from BaseException

In [28]:


final_routes['score_cumulative_avg'] = final_routes.groupby(['driver_id_sorted', 'day_of_week'])['label'].transform(
        lambda x: x.expanding().mean()
    )
final_routes['score_cumulative_driver'] = final_routes.groupby(['driver_id_sorted'])['label'].transform(
        lambda x: x.expanding().mean()
    )
final_routes['score_avg_total'] = final_routes['label'].expanding().mean()

final_routes

,routes,actual_routes,driver_id,distance_route,distance_actual_route,last_two_weeks_count,current_lat,current_lng,day_of_week,date,...,day_of_week_encoded,experience_feature,len_feature,driver_id_feature,country_flag_feature,day_of_week_feature,day_of_week_encoded_ext,score_cumulative_avg,score_cumulative_driver,score_avg_total
0,"[1, 1385, 1, 1386, 1387, 49, 1388, 50, 9229, 4...","[1, 1385, 1, 1386, 1387, 49, 1388, 50, 9229, 4...",1184,"[23.479040480510506, 23.479040480510506, 26.97...","[0.0, 23.479040480510506, 8.146314511821483, 3...",8,"[55.64641920439919, 55.32612200000001, 55.6464...","[12.15372974969827, 11.9560479, 12.15372974969...",Monday,2024-01-15,...,2,"[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]","[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 1...","[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[0, 1, 0, 0, 0, 0, 0]",1.000000,1.000000,1.000000
1,"[1, 1171, 1172, 1173, 1954, 1954, 4762, 1183, ...","[1, 1171, 1172, 1173, 1954, 1954, 4762, 1183, ...",1486,"[1.141374091100897, 2.8522152887994903, 1.5389...","[1.141374091100897, 2.8522152887994903, 1.5389...",2,"[55.64641920439919, 55.649076, 55.63469, 55.63...","[12.15372974969827, 12.124935, 12.05662, 12.01...",Monday,2024-01-15,...,2,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]","[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...","[146, 146, 146, 146, 146, 146, 146, 146, 146, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[0, 1, 0, 0, 0, 0, 0]",1.000000,1.000000,1.000000
2,"[287, 287, 288, 289, 290, 291, 292, 293, 294, ...","[287, 287, 288, 289, 290, 291, 292, 293, 294, ...",1432,"[0.0, 16.752201025499367, 1.501855696871057, 0...","[0.0, 16.752201025499367, 1.501855696871057, 0...",8,"[59.805664, 59.805664, 59.910491, 59.922304900...","[10.290383, 10.290383, 10.723998, 10.6877597, ...",Monday,2024-01-15,...,2,"[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]","[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...","[23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[0, 1, 0, 0, 0, 0, 0]",0.000000,0.000000,0.666667
3,"[287, 1419, 1420, 1420, 1421, 1422, 1423, 287]","[287, 1419, 1420, 1420, 1421, 1422, 1423, 287]",723,"[31.65996164052381, 0.43893999280403884, 0.0, ...","[31.65996164052381, 0.43893999280403884, 0.0, ...",4,"[59.805664, 60.066484, 60.072593, 60.072593, 6...","[10.290383, 11.039009, 11.035613, 11.035613, 1...",Monday,2024-01-15,...,2,"[4, 4, 4, 4, 4, 4, 4, 4]","[8, 8, 8, 8, 8, 8, 8, 8]","[113, 113, 113, 113, 113, 113, 113, 113]","[0, 0, 0, 0, 0, 0, 0, 0]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[0, 1, 0, 0, 0, 0, 0]",0.000000,0.000000,0.500000
4,"[1, 3787, 3785, 3784, 3781, 3780, 3782, 3783, ...","[1, 3787, 3785, 3784, 3781, 3780, 3782, 3783, ...",1324,"[8.678113362175365, 0.7124995743679728, 0.2457...","[10.893805973040553, 3.7200180786093084, 0.245...",4,"[55.64641920439919, 55.77168, 55.766721, 55.76...","[12.15372974969827, 12.165606, 12.181618, 12.1...",Monday,2024-01-15,...,2,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]","[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...","[27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[0, 1, 0, 0, 0, 0, 0]",1.000000,1.000000,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18557,"[1, 4270, 5391, 3861, 4651, 2736, 4652, 2734, ...","[1, 4270, 5391, 3861, 4651, 2736, 4652, 2734, ...",1243,"[68.89851202059795, 1.2509793817380201, 4.9521...","[68.89851202059795, 1.2509793817380201, 4.9521...",1,"[55.64641920439919, 54.696715, 54.6986631, 54....","[12.15372974969827, 11.629326, 11.598282, 11.4...",Thursday,2024-08-07,...,5,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11]","[17, 17, 17, 17, 17, 

# Average benchmark here

In [29]:
# result = final_routes.groupby('driver_id_sorted')[['driver_id_sorted', 'score']].apply(
#     lambda x: pd.Series({'count': len(x), 'score': x['score'].tolist()})
# ).reset_index()
#
# sorted_result = result.sort_values(by='count', ascending=False).reset_index(drop=True)


In [30]:
import pandas as pd
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score, log_loss, brier_score_loss
)
import matplotlib.pyplot as plt

def get_predictions(df, scores_dict):
    def predict(row):
        # Try to get the score from the main dictionary
        label = scores_dict.get((row['driver_id_sorted'], row['day_of_week']))

        # If not found, fall back to the driver-only dictionary
        if label is None:
            print('1')
            return 0.5
        else:
            return label

    return df.apply(predict, axis=1)


# Function to calculate metrics
def calculate_metrics(y_true, y_pred, y_pred_proba):
    acc = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_pred)
    average_precision = average_precision_score(y_true, y_pred)
    quadratic_loss = log_loss(y_true, y_pred_proba)
    brier_score = brier_score_loss(y_true, y_pred_proba)

    metrics = {
        "acc": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "roc_auc": roc_auc,
        "average_precision": average_precision,
        "quadratic_loss": quadratic_loss,
        "brier_score": brier_score,
    }
    return metrics

In [31]:
from sklearn.metrics import confusion_matrix
def get_predictions(df, scores_dict):
    def predict(row):
        # Try to get the score from the main dictionary
        label = scores_dict.get((row['driver_id_sorted'], row['day_of_week']))

        # If not found, fall back to the driver-only dictionary
        if label is None:
            print('1')
            return 0.5
        else:
            return label

    return df.apply(predict, axis=1)
def calculate_metrics(y_true, y_pred, y_pred_proba):
    acc = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_pred)
    average_precision = average_precision_score(y_true, y_pred)
    quadratic_loss = log_loss(y_true, y_pred_proba)
    brier_score = brier_score_loss(y_true, y_pred_proba)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    metrics = {
        "acc": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "roc_auc": roc_auc,
        "average_precision": average_precision,
        "quadratic_loss": quadratic_loss,
        "brier_score": brier_score,
        "true_positive": tp,
        "false_positive": fp,
        "true_negative": tn,
        "false_negative": fn
    }
    return metrics

def split_group(group):
    n = len(group)
    if n < 4:  # Ensure at least 1 sample for train and 1 each for val and test
        return pd.DataFrame()

    # Calculate sizes for each split
    train_size = max(int(n * 0.8), n - 4)  # Ensure at least 2 samples total for val and test
    val_test_size = n - train_size

    # Always give the extra sample to val when odd
    val_size = (val_test_size + 1) // 2  # This will be larger when val_test_size is odd
    test_size = val_test_size - val_size

    return pd.concat([
        group.iloc[:train_size].assign(split='train'),
        group.iloc[train_size:train_size+val_size].assign(split='val'),
        group.iloc[train_size+val_size:].assign(split='test')
    ])

def categorize_predictions(df, y_true, y_pred, y_pred_proba):
    df = df.copy()
    df['prediction_category'] = 'Unknown'
    df.loc[(y_true == 1) & (y_pred == 1), 'prediction_category'] = 'True Positive'
    df.loc[(y_true == 0) & (y_pred == 1), 'prediction_category'] = 'False Positive'
    df.loc[(y_true == 1) & (y_pred == 0), 'prediction_category'] = 'False Negative'
    df.loc[(y_true == 0) & (y_pred == 0), 'prediction_category'] = 'True Negative'
    df['predictive_probability'] = y_pred_proba
    return df

def save_categorized_routes(df, filename):
    df.to_csv(filename, index=False)
    print(f"Saved categorized routes to {filename}")


final_routes_split = final_routes.groupby(['driver_id_sorted', 'day_of_week']).apply(split_group).reset_index(drop=True)
# Separate into train, validation, and test sets

drivers_dic = {}
k = 1
for driver in final_routes_split['driver_id_sorted']:
    if driver not in drivers_dic:
        drivers_dic[driver] = k
        k += 1
print('Total number of drivers', len(drivers_dic))
total_drivers = len(drivers_dic)
encoding_drivers = []
for driver in final_routes_split['driver_id_sorted']:
    encoding_drivers.append(drivers_dic[driver])
#
final_routes_split['driver_id_sorted'] = encoding_drivers

locations_dic = {}
location_count = {}
k = 1
for row in final_routes_split['routes']:
   for location in row:
       if location not in locations_dic:
           locations_dic[location] = k
           k += 1
k = 1
print(len(locations_dic))

for row in final_routes_split['routes']:
   for location in row:
       if location not in location_count:
            location_count[location] = 1
       else:
            location_count[location] += 1
encoding_routes = []
for row in final_routes_split['routes']:
    encoding_route = []
    for location in row:
        encoding_route.append(locations_dic[location])
    encoding_routes.append(encoding_route)
final_routes_split['routes'] = encoding_routes

print(len(final_routes_split))
train_df = final_routes_split[final_routes_split['split'] == 'train'].drop('split', axis=1)
val_df = final_routes_split[final_routes_split['split'] == 'val'].drop('split', axis=1)
test_df = final_routes_split[final_routes_split['split'] == 'test'].drop('split', axis=1)

print('lens', len(train_df), len(val_df), len(test_df))
combined_df = pd.concat([train_df, val_df, test_df], axis=0)
combined_df = combined_df.reset_index(drop=True)

val_y_true = val_df['label']
average_scores_dict = train_df.groupby(['driver_id_sorted', 'day_of_week'])['score_cumulative_avg'].last().to_dict()
average_scores_dict_driver = train_df.groupby(['driver_id_sorted'])['score_cumulative_driver'].last().to_dict()
total_avg = train_df['score_avg_total'].iloc[-1]
val_y_pred_proba = get_predictions(val_df, average_scores_dict)
val_y_pred = (val_y_pred_proba.values > 0.5).astype(int)
val_metrics = calculate_metrics(val_y_true, val_y_pred, val_y_pred_proba)

# # # Calculate metrics for test set
test_y_true = test_df['label']
test_y_pred_proba = get_predictions(test_df, average_scores_dict)
test_y_pred = (test_y_pred_proba.values > 0.5).astype(int)
test_metrics = calculate_metrics(test_y_true, test_y_pred, test_y_pred_proba)

# test_y_pred_proba = pd.Series([total_avg] * len(test_y_true))
# test_y_pred = (test_y_pred_proba > 0.5).astype(int)
# test_metrics = calculate_metrics(test_y_true, test_y_pred, test_y_pred_proba)

print('val_metrics', val_metrics)
print('test_metrics', test_metrics)

test_df_group_HA = categorize_predictions(test_df, test_y_true, test_y_pred, test_y_pred_proba)
# save_categorized_routes(test_df_categorized, 'test_routes_categorized_HA.csv')
test_df_group_HA[test_df_group_HA['prediction_category'] == 'True Positive']['routes']
print(len(test_df_group_HA[test_df_group_HA['prediction_category'] == 'True Positive']['routes']), len(test_df_group_HA[test_df_group_HA['prediction_category'] == 'False Positive']['routes']))

Total number of drivers 314
10546
17667
lens 13823 2188 1656
val_metrics {'acc': 0.7079524680073126, 'precision': 0.7094240837696335, 'recall': 0.7265415549597856, 'f1': 0.7178807947019867, 'roc_auc': 0.7075177372553839, 'average_precision': 0.6552798246627187, 'quadratic_loss': 1.3110032629733819, 'brier_score': 0.1963393520342656, 'true_positive': 813, 'false_positive': 333, 'true_negative': 736, 'false_negative': 306}
test_metrics {'acc': 0.7137681159420289, 'precision': 0.7096774193548387, 'recall': 0.7350835322195705, 'f1': 0.7221570926143025, 'roc_auc': 0.7135075362809344, 'average_precision': 0.655730155170317, 'quadratic_loss': 1.3141721524216174, 'brier_score': 0.19706852026087537, 'true_positive': 616, 'false_positive': 252, 'true_negative': 566, 'false_negative': 222}
616 252


/var/folders/0j/mpbqwzf51p38zw2w423xp1fw0000gn/T/ipykernel_83336/1217222917.py:76: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  final_routes_split = final_routes.groupby(['driver_id_sorted', 'day_of_week']).apply(split_group).reset_index(drop=True)


# After benchmark, continue model

In [32]:
final_routes_split

,routes,actual_routes,driver_id,distance_route,distance_actual_route,last_two_weeks_count,current_lat,current_lng,day_of_week,date,...,experience_feature,len_feature,driver_id_feature,country_flag_feature,day_of_week_feature,day_of_week_encoded_ext,score_cumulative_avg,score_cumulative_driver,score_avg_total,split
0,"[1, 2, 3, 4, 5, 6, 6, 7, 8]","[1, 2, 3, 156, 4, 5, 5, 6, 7]",1255.0,"[14.640018023192697, 1.5229312527257717, 0.185...","[14.640018023192697, 1.7084597707961175, 0.185...",5.0,"[55.64641920439919, 55.6669147, 55.6757322, 55...","[12.15372974969827, 12.5262801, 12.5619711, 12...",Friday,2024-01-18,...,"[5, 5, 5, 5, 5, 5, 5, 5, 5]","[9, 9, 9, 9, 9, 9, 9, 9, 9]","[1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[1, 0, 0, 0, 0, 0, 0]",0.000000,0.250000,0.607038,train
1,"[1, 6, 6, 1, 2, 3, 4, 5, 7, 8]","[1, 5, 5, 1, 2, 3, 156, 4, 6, 7]",1255.0,"[16.153537949626582, 0.0, 16.153537949626582, ...","[16.274521631876873, 0.18579849611701657, 0.70...",4.0,"[55.64641920439919, 55.6657227, 55.6657227, 55...","[12.15372974969827, 12.5653169, 12.5653169, 12...",Friday,2024-01-25,...,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4]","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[1, 0, 0, 0, 0, 0, 0]",0.500000,0.285714,0.591422,train
2,"[1, 2, 3, 4, 5, 6, 6, 7, 8]","[1, 2, 3, 156, 4, 5, 5, 6, 7]",1255.0,"[14.640018023192697, 1.5229312527257717, 0.185...","[14.640018023192697, 1.7084597707961175, 0.185...",6.0,"[55.64641920439919, 55.6669147, 55.6757322, 55...","[12.15372974969827, 12.5262801, 12.5619711, 12...",Friday,2024-02-08,...,"[6, 6, 6, 6, 6, 6, 6, 6, 6]","[9, 9, 9, 9, 9, 9, 9, 9, 9]","[1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[1, 0, 0, 0, 0, 0, 0]",0.333333,0.200000,0.564024,train
3,"[1, 2, 9, 4, 5, 6, 7, 8]","[1, 2, 14, 156, 4, 5, 6, 7]",1255.0,"[14.640018023192697, 1.400306417188586, 0.4137...","[14.640018023192697, 1.400306417188586, 0.4137...",11.0,"[55.64641920439919, 55.6669147, 55.671473, 55....","[12.15372974969827, 12.5262801, 12.561174, 12....",Friday,2024-02-22,...,"[11, 11, 11, 11, 11, 11, 11, 11]","[8, 8, 8, 8, 8, 8, 8, 8]","[1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[1, 0, 0, 0, 0, 0, 0]",0.250000,0.200000,0.559459,train
4,"[1, 1, 2, 9, 3, 4, 5, 6, 7, 10, 8]","[1, 1, 2, 14, 3, 156, 4, 5, 6, 3769, 7]",1255.0,"[0.0, 14.640018023192697, 1.400306417188586, 0...","[0.0, 14.640018023192697, 1.400306417188586, 0...",11.0,"[55.64641920439919, 55.64641920439919, 55.6669...","[12.15372974969827, 12.15372974969827, 12.5262...",Friday,2024-02-29,...,"[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11]","[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[1, 0, 0, 0, 0, 0, 0]",0.200000,0.250000,0.554858,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17662,"[8656, 9169, 9170, 9180, 9177, 9171, 9174, 917...","[4315, 9711, 9715, 9717, 9738, 9712, 9714, 971...",1555.0,"[10.550783573784587, 0.025338060555759063, 0.1...","[10.649090169527625, 0.036295189300920946, 0.0...",9.0,"[56.0667627, 56.159131, 56.15884339999999, 56....","[9.986944, 10.204122, 10.2037157, 10.1991044, ...",Wednesday,2024-05-21,...,"[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, ...","[20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 2...","[387, 387, 387, 387, 387, 387, 387, 387, 387, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[0, 0, 0, 0, 0, 0, 1]",1.000000,1.000000,0.542148,train
17663,"[8656, 9169, 9170, 9183, 9180, 9171, 9174, 917...","[4315, 9711, 9715, 9710, 9717, 9712, 9714, 971...",1555.0,"[10.550783573784587, 0.025338060555759063, 0.2...","[

In [33]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Assuming final_routes dataframe is available and sorted by date
# If not, make sure to sort it by date first

# Prepare the data
rf_df = final_routes_split[['driver_id_sorted', 'day_of_week_encoded', 'label', 'split', 'routes']]
o_day_of = pd.get_dummies(rf_df['day_of_week_encoded'], prefix='day')
o_driver = pd.get_dummies(rf_df['driver_id_sorted'], prefix='driver')
rf_df = pd.concat([o_day_of, o_driver, rf_df['split'], rf_df['label'], rf_df['routes']], axis=1)


for col in rf_df.columns:
    if col != 'split' and col != 'label' and col != 'routes':
        rf_df[col] = rf_df[col].astype(int)

train_df = rf_df[rf_df['split'] == 'train'].drop('split', axis=1)
val_df = rf_df[rf_df['split'] == 'val'].drop('split', axis=1)
test_df = rf_df[rf_df['split'] == 'test'].drop('split', axis=1)

# Create X_train and y_train
X_train = train_df.drop('label', axis=1)
y_train = train_df['label']

# Create X_val and y_val (optional, but often useful)
X_val = val_df.drop('label', axis=1)
y_val = val_df['label']

# Create X_test and y_test (optional, but often useful)
X_test = test_df.drop('label', axis=1)
y_test = test_df['label']

print(len(train_df), len(val_df), len(test_df))
#
#
# Train the model
model = RandomForestRegressor(random_state=42)
X_trainwr = X_train.drop('routes', axis=1)
model.fit(X_trainwr, y_train)

# Function to calculate metrics
def calculate_metrics(y_true, y_pred, y_pred_proba):
    acc = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_pred)
    average_precision = average_precision_score(y_true, y_pred)
    quadratic_loss = log_loss(y_true, y_pred_proba)
    brier_score = brier_score_loss(y_true, y_pred_proba)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    metrics = {
        "acc": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "roc_auc": roc_auc,
        "average_precision": average_precision,
        "quadratic_loss": quadratic_loss,
        "brier_score": brier_score,
        "true_positive": tp,
        "false_positive": fp,
        "true_negative": tn,
        "false_negative": fn
    }
    return metrics

# Evaluate on validation set
y_val_pred_proba = model.predict(X_val.drop('routes', axis=1))
y_val_pred = (y_val_pred_proba > 0.5).astype(int)
val_metrics = calculate_metrics(y_val, y_val_pred, y_val_pred_proba)

y_test_pred_proba = model.predict(X_test.drop('routes', axis=1))
y_test_pred = (y_test_pred_proba > 0.5).astype(int)
test_metrics = calculate_metrics(y_test, y_test_pred, y_test_pred_proba)

print(val_metrics)
print(test_metrics)

test_df_group_RF = categorize_predictions(test_df, y_test, y_test_pred, y_test_pred_proba)
test_df_group_RF
test_df

13823 2188 1656
{'acc': 0.7143510054844607, 'precision': 0.7061769616026711, 'recall': 0.7560321715817694, 'f1': 0.7302546396201985, 'roc_auc': 0.713376235463476, 'average_precision': 0.6586639826058708, 'quadratic_loss': 1.125556317746882, 'brier_score': 0.19601517330062207, 'true_positive': 846, 'false_positive': 352, 'true_negative': 717, 'false_negative': 273}
{'acc': 0.7125603864734299, 'precision': 0.7002212389380531, 'recall': 0.7553699284009546, 'f1': 0.7267508610792193, 'roc_auc': 0.7120370424400861, 'average_precision': 0.6527183376528659, 'quadratic_loss': 1.2010726563682181, 'brier_score': 0.19659934001855467, 'true_positive': 633, 'false_positive': 271, 'true_negative': 547, 'false_negative': 205}


,day_1.0,day_2.0,day_3.0,day_4.0,day_5.0,day_6.0,day_7.0,driver_1,driver_2,driver_3,...,driver_307,driver_308,driver_309,driver_310,driver_311,driver_312,driver_313,driver_314,label,routes
7,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1.0,"[1, 10, 11, 9, 3, 4, 5, 6, 6, 7, 8]"
26,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1.0,"[1, 17, 5, 6, 6, 10, 8, 7, 3, 4]"
27,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0.0,"[1, 17, 10, 6, 6, 5, 4, 7, 8]"
38,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1.0,"[1, 13, 10, 17, 9, 3, 4, 5, 6, 6, 7, 8]"
55,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1.0,"[1, 1, 17, 10, 5, 4, 3, 9, 6, 6, 7, 8]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17623,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1.0,"[8656, 9169, 9170, 9177, 9171, 9221, 9175, 917..."
17633,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1.0,"[8656, 9179, 9171, 9174, 9175, 9221, 9177, 916..."
17648,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1.0,"[8656, 9179, 9208, 9180, 9171, 9177, 9169, 917..."
17657,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,1.0,"[8656, 9179, 9180, 9208, 9169, 9177, 9171, 917..."


In [34]:
test_df_group_RF[test_df_group_RF['prediction_category'] == 'True Positive']['routes']

26                        [1, 17, 5, 6, 6, 10, 8, 7, 3, 4]
38                 [1, 13, 10, 17, 9, 3, 4, 5, 6, 6, 7, 8]
55                  [1, 1, 17, 10, 5, 4, 3, 9, 6, 6, 7, 8]
56               [1, 6, 9, 3, 4, 17, 10, 5, 5, 7, 8, 1, 6]
74                        [1, 17, 10, 6, 6, 5, 9, 3, 7, 8]
                               ...                        
17623    [8656, 9169, 9170, 9177, 9171, 9221, 9175, 917...
17633    [8656, 9179, 9171, 9174, 9175, 9221, 9177, 916...
17648    [8656, 9179, 9208, 9180, 9171, 9177, 9169, 917...
17657    [8656, 9179, 9180, 9208, 9169, 9177, 9171, 917...
17666    [8656, 9179, 9169, 9171, 9171, 9174, 9175, 917...
Name: routes, Length: 633, dtype: object

In [35]:
ha_tp_routes = test_df_group_HA[test_df_group_HA['prediction_category'] == 'False Positive']['routes'].tolist()
rf_tp_routes = test_df_group_RF[test_df_group_RF['prediction_category'] == 'False Positive']['routes'].tolist()

# Convert lists to sets of tuples for comparison
ha_tp_routes_set = set(tuple(route) for route in ha_tp_routes)
rf_tp_routes_set = set(tuple(route) for route in rf_tp_routes)

# Find non-overlapping routes
ha_only_routes = ha_tp_routes_set - rf_tp_routes_set
rf_only_routes = rf_tp_routes_set - ha_tp_routes_set

# Convert back to lists for display
ha_only_routes = [list(route) for route in ha_only_routes]
rf_only_routes = [list(route) for route in rf_only_routes]

print("Routes unique to HA model:", len(ha_only_routes))
print("Sample of HA-only routes:", ha_only_routes[:5])

print("\nRoutes unique to RF model:", len(rf_only_routes))
print("Sample of RF-only routes:", rf_only_routes[:5])

Routes unique to HA model: 0
Sample of HA-only routes: []

Routes unique to RF model: 19
Sample of RF-only routes: [[1, 1, 224, 1522, 66, 66, 34, 6420, 65, 222], [1, 5162, 1054, 259, 1055, 1], [1272, 1652, 1653, 1654, 1879, 1685, 1686, 1878, 1849, 2111, 1264, 7513], [1, 1026, 1026, 1514, 1515, 294, 1002, 680, 680, 1043, 339, 275], [1, 722, 731, 752, 733, 760, 736, 727, 735, 728, 739, 364]]


In [36]:
import torch
import torch.nn as nn
import random
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pandas as pd

# Assuming final_routes dataframe and relevant imports are available
def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


set_seed(42)
torch.use_deterministic_algorithms(True)

class SimpleNN(nn.Module):
    def __init__(self, input_size):
        super(SimpleNN, self).__init__()
        self.layer1 = nn.Linear(input_size, 64)
        self.layer2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.layer2(x)
        x = self.sigmoid(x)
        return x

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train.drop('routes', axis=1).values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_val_tensor = torch.tensor(X_val.drop('routes', axis=1).values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test.drop('routes', axis=1).values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Training settings
num_epochs = 100
learning_rate = 0.001

model = SimpleNN(X_train.shape[1]-1)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training and validation loop
best_val_loss = float('inf')
best_model = None

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        val_loss = criterion(val_outputs, y_val_tensor)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model.state_dict()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

# Load the best model
model.load_state_dict(best_model)

# Evaluation on test set
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    y_true = y_test_tensor

y_pred = y_pred.numpy()
y_true = y_true.numpy()

y_test_pred = (y_pred > 0.5).astype(int)
test_metrics = calculate_metrics(y_test, y_test_pred, y_pred)

print(test_metrics)

y_pred = y_pred.flatten()
y_test_pred = y_test_pred.flatten()

test_df_group_NN = categorize_predictions(X_test, y_test, y_test_pred, y_pred)
test_df_group_NN

Epoch [10/100], Train Loss: 0.6935, Val Loss: 0.6904
Epoch [20/100], Train Loss: 0.6823, Val Loss: 0.6831
Epoch [30/100], Train Loss: 0.6679, Val Loss: 0.6734
Epoch [40/100], Train Loss: 0.6496, Val Loss: 0.6609
Epoch [50/100], Train Loss: 0.6279, Val Loss: 0.6454
Epoch [60/100], Train Loss: 0.6041, Val Loss: 0.6274
Epoch [70/100], Train Loss: 0.5801, Val Loss: 0.6091
Epoch [80/100], Train Loss: 0.5582, Val Loss: 0.5935
Epoch [90/100], Train Loss: 0.5400, Val Loss: 0.5817
Epoch [100/100], Train Loss: 0.5258, Val Loss: 0.5736
{'acc': 0.7083333333333334, 'precision': 0.6839378238341969, 'recall': 0.7875894988066826, 'f1': 0.7321131447587355, 'roc_auc': 0.7073644315549306, 'average_precision': 0.6461501705938223, 'quadratic_loss': 0.5722858352739434, 'brier_score': 0.19321791756549994, 'true_positive': 660, 'false_positive': 305, 'true_negative': 513, 'false_negative': 178}


,day_1.0,day_2.0,day_3.0,day_4.0,day_5.0,day_6.0,day_7.0,driver_1,driver_2,driver_3,...,driver_308,driver_309,driver_310,driver_311,driver_312,driver_313,driver_314,routes,prediction_category,predictive_probability
7,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,"[1, 10, 11, 9, 3, 4, 5, 6, 6, 7, 8]",True Positive,0.726546
26,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,"[1, 17, 5, 6, 6, 10, 8, 7, 3, 4]",True Positive,0.706859
27,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,"[1, 17, 10, 6, 6, 5, 4, 7, 8]",False Positive,0.706859
38,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,"[1, 13, 10, 17, 9, 3, 4, 5, 6, 6, 7, 8]",True Positive,0.805362
55,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,"[1, 1, 17, 10, 5, 4, 3, 9, 6, 6, 7, 8]",True Positive,0.711726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17623,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,"[8656, 9169, 9170, 9177, 9171, 9221, 9175, 917...",True Positive,0.796079
17633,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,"[8656, 9179, 9171, 9174, 9175, 9221, 9177, 916...",True Positive,0.781804
17648,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,"[8656, 9179, 9208, 9180, 9171, 9177, 9169, 917...",True Positive,0.785387
17657,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,"[8656, 9179, 9180, 9208, 9169, 9177, 9171, 917...",True Positive,0.789889


In [37]:
ha_tp_routes = test_df_group_HA[test_df_group_HA['prediction_category'] == 'False Positive']['routes'].tolist()
nn_tp_routes = test_df_group_NN[test_df_group_NN['prediction_category'] == 'False Positive']['routes'].tolist()

# Convert lists to sets of tuples for comparison
ha_tp_routes_set = set(tuple(route) for route in ha_tp_routes)
nn_tp_routes_set = set(tuple(route) for route in nn_tp_routes)

# Find non-overlapping routes
ha_only_routes = ha_tp_routes_set - nn_tp_routes_set
nn_only_routes = nn_tp_routes_set - ha_tp_routes_set

# Convert back to lists for display
ha_only_routes = [list(route) for route in ha_only_routes]
nn_only_routes = [list(route) for route in nn_only_routes]

print("Routes unique to HA model:", len(ha_only_routes))
print("Sample of HA-only routes:", ha_only_routes[:5])

print("\nRoutes unique to RF model:", len(nn_only_routes))
print("Sample of RF-only routes:", nn_only_routes[:5])

Routes unique to HA model: 29
Sample of HA-only routes: [[1272, 1752, 1864, 1786, 1788, 1564, 1851, 1570], [1162, 4385, 5696, 5696, 4360, 1162, 1162], [1, 1, 7431, 5001, 7201, 1144, 1144, 7202, 7437, 1148], [1272, 1573, 1573, 1574, 1272, 1633, 1813, 1777], [2116, 2141, 2174, 2164, 2781, 2148, 2498, 2271, 2413, 2274, 2410, 2252]]

Routes unique to RF model: 81
Sample of RF-only routes: [[1162, 5715, 7072, 7072, 4511, 5716, 5794, 5702, 7099, 7105], [1162, 3700, 3701, 6963, 3621, 3622, 3624, 3627, 3629, 3630], [1, 1386, 1399, 1400, 1401, 1382, 1380, 1381, 1407, 1388, 1405, 1389], [1, 991, 990, 741, 694, 1027, 1043, 311, 4993, 1050, 1050, 5160, 674, 6473, 281, 1005, 1], [1272, 1633, 1809, 1833]]


In [38]:
# unique_days = final_routes['day_of_week'].unique()
# day_of_week_encoded = pd.get_dummies(final_routes['day_of_week'], prefix='day_of_week')
# final_routes = pd.concat([final_routes, day_of_week_encoded], axis=1)
# final_routes
print(len(y_pred), len(y_pred[y_pred>0.5]))
print(len(y_test), len(y_test[y_test>0.5]))

1656 965
1656 838


# Model

In [39]:
def padding_(routes, route_len):
    features = np.zeros((len(routes), route_len),dtype=np.float16)
    for ii, route in enumerate(routes):
        if len(route) != 0:
            features[ii, -len(route):] = np.array(route)[:route_len]
    return features

X = final_routes_split.drop(columns = ['driver_id', 'len', 'driver_id_sorted','day_of_week_encoded'])
max_route_length = max(len(item) for item in final_routes_split['routes'])
y = np.array(final_routes_split['label'])
# X = np.concatenate([padding_(X['routes'], max_route_length),padding_(X['distance_route'], max_route_length), X.to_numpy()[:,2:]], axis=1)
# X = np.concatenate([padding_(X['routes'], max_route_length),padding_(X['distance_route'], max_route_length), padding_(X['experience_feature'], max_route_length), padding_(X['len_feature'], max_route_length), padding_(X['driver_id_feature'], max_route_length)], axis=1)
# X = X.astype(np.float16)
# X = X.astype(int) #for boolean values, to converst from string to int
# final_routes
final_routes_split

,routes,actual_routes,driver_id,distance_route,distance_actual_route,last_two_weeks_count,current_lat,current_lng,day_of_week,date,...,experience_feature,len_feature,driver_id_feature,country_flag_feature,day_of_week_feature,day_of_week_encoded_ext,score_cumulative_avg,score_cumulative_driver,score_avg_total,split
0,"[1, 2, 3, 4, 5, 6, 6, 7, 8]","[1, 2, 3, 156, 4, 5, 5, 6, 7]",1255.0,"[14.640018023192697, 1.5229312527257717, 0.185...","[14.640018023192697, 1.7084597707961175, 0.185...",5.0,"[55.64641920439919, 55.6669147, 55.6757322, 55...","[12.15372974969827, 12.5262801, 12.5619711, 12...",Friday,2024-01-18,...,"[5, 5, 5, 5, 5, 5, 5, 5, 5]","[9, 9, 9, 9, 9, 9, 9, 9, 9]","[1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[1, 0, 0, 0, 0, 0, 0]",0.000000,0.250000,0.607038,train
1,"[1, 6, 6, 1, 2, 3, 4, 5, 7, 8]","[1, 5, 5, 1, 2, 3, 156, 4, 6, 7]",1255.0,"[16.153537949626582, 0.0, 16.153537949626582, ...","[16.274521631876873, 0.18579849611701657, 0.70...",4.0,"[55.64641920439919, 55.6657227, 55.6657227, 55...","[12.15372974969827, 12.5653169, 12.5653169, 12...",Friday,2024-01-25,...,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4]","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[1, 0, 0, 0, 0, 0, 0]",0.500000,0.285714,0.591422,train
2,"[1, 2, 3, 4, 5, 6, 6, 7, 8]","[1, 2, 3, 156, 4, 5, 5, 6, 7]",1255.0,"[14.640018023192697, 1.5229312527257717, 0.185...","[14.640018023192697, 1.7084597707961175, 0.185...",6.0,"[55.64641920439919, 55.6669147, 55.6757322, 55...","[12.15372974969827, 12.5262801, 12.5619711, 12...",Friday,2024-02-08,...,"[6, 6, 6, 6, 6, 6, 6, 6, 6]","[9, 9, 9, 9, 9, 9, 9, 9, 9]","[1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[1, 0, 0, 0, 0, 0, 0]",0.333333,0.200000,0.564024,train
3,"[1, 2, 9, 4, 5, 6, 7, 8]","[1, 2, 14, 156, 4, 5, 6, 7]",1255.0,"[14.640018023192697, 1.400306417188586, 0.4137...","[14.640018023192697, 1.400306417188586, 0.4137...",11.0,"[55.64641920439919, 55.6669147, 55.671473, 55....","[12.15372974969827, 12.5262801, 12.561174, 12....",Friday,2024-02-22,...,"[11, 11, 11, 11, 11, 11, 11, 11]","[8, 8, 8, 8, 8, 8, 8, 8]","[1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[1, 0, 0, 0, 0, 0, 0]",0.250000,0.200000,0.559459,train
4,"[1, 1, 2, 9, 3, 4, 5, 6, 7, 10, 8]","[1, 1, 2, 14, 3, 156, 4, 5, 6, 3769, 7]",1255.0,"[0.0, 14.640018023192697, 1.400306417188586, 0...","[0.0, 14.640018023192697, 1.400306417188586, 0...",11.0,"[55.64641920439919, 55.64641920439919, 55.6669...","[12.15372974969827, 12.15372974969827, 12.5262...",Friday,2024-02-29,...,"[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11]","[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[1, 0, 0, 0, 0, 0, 0]",0.200000,0.250000,0.554858,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17662,"[8656, 9169, 9170, 9180, 9177, 9171, 9174, 917...","[4315, 9711, 9715, 9717, 9738, 9712, 9714, 971...",1555.0,"[10.550783573784587, 0.025338060555759063, 0.1...","[10.649090169527625, 0.036295189300920946, 0.0...",9.0,"[56.0667627, 56.159131, 56.15884339999999, 56....","[9.986944, 10.204122, 10.2037157, 10.1991044, ...",Wednesday,2024-05-21,...,"[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, ...","[20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 2...","[387, 387, 387, 387, 387, 387, 387, 387, 387, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[0, 0, 0, 0, 0, 0, 1]",1.000000,1.000000,0.542148,train
17663,"[8656, 9169, 9170, 9183, 9180, 9171, 9174, 917...","[4315, 9711, 9715, 9710, 9717, 9712, 9714, 971...",1555.0,"[10.550783573784587, 0.025338060555759063, 0.2...","[

# Driver id and location id counting and overwritten with transformed dataset

In [40]:
drivers_dic = {}
k = 1
for driver in final_routes_split['driver_id_sorted']:
    if driver not in drivers_dic:
        drivers_dic[driver] = k
        k += 1
print('Total number of drivers', len(drivers_dic))
total_drivers = len(drivers_dic)
encoding_drivers = []
for driver in final_routes_split['driver_id_sorted']:
    encoding_drivers.append(drivers_dic[driver])
#
final_routes_split['driver_id_sorted'] = encoding_drivers

locations_dic = {}
location_count = {}
k = 1
for row in final_routes_split['routes']:
   for location in row:
       if location not in locations_dic:
           locations_dic[location] = k
           k += 1
k = 1
print(len(locations_dic))

for row in final_routes_split['routes']:
   for location in row:
       if location not in location_count:
            location_count[location] = 1
       else:
            location_count[location] += 1
encoding_routes = []
for row in final_routes_split['routes']:
    encoding_route = []
    for location in row:
        encoding_route.append(locations_dic[location])
    encoding_routes.append(encoding_route)
final_routes_split['routes'] = encoding_routes

final_routes_split

Total number of drivers 314
10546


,routes,actual_routes,driver_id,distance_route,distance_actual_route,last_two_weeks_count,current_lat,current_lng,day_of_week,date,...,experience_feature,len_feature,driver_id_feature,country_flag_feature,day_of_week_feature,day_of_week_encoded_ext,score_cumulative_avg,score_cumulative_driver,score_avg_total,split
0,"[1, 2, 3, 4, 5, 6, 6, 7, 8]","[1, 2, 3, 156, 4, 5, 5, 6, 7]",1255.0,"[14.640018023192697, 1.5229312527257717, 0.185...","[14.640018023192697, 1.7084597707961175, 0.185...",5.0,"[55.64641920439919, 55.6669147, 55.6757322, 55...","[12.15372974969827, 12.5262801, 12.5619711, 12...",Friday,2024-01-18,...,"[5, 5, 5, 5, 5, 5, 5, 5, 5]","[9, 9, 9, 9, 9, 9, 9, 9, 9]","[1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[1, 0, 0, 0, 0, 0, 0]",0.000000,0.250000,0.607038,train
1,"[1, 6, 6, 1, 2, 3, 4, 5, 7, 8]","[1, 5, 5, 1, 2, 3, 156, 4, 6, 7]",1255.0,"[16.153537949626582, 0.0, 16.153537949626582, ...","[16.274521631876873, 0.18579849611701657, 0.70...",4.0,"[55.64641920439919, 55.6657227, 55.6657227, 55...","[12.15372974969827, 12.5653169, 12.5653169, 12...",Friday,2024-01-25,...,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4]","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[1, 0, 0, 0, 0, 0, 0]",0.500000,0.285714,0.591422,train
2,"[1, 2, 3, 4, 5, 6, 6, 7, 8]","[1, 2, 3, 156, 4, 5, 5, 6, 7]",1255.0,"[14.640018023192697, 1.5229312527257717, 0.185...","[14.640018023192697, 1.7084597707961175, 0.185...",6.0,"[55.64641920439919, 55.6669147, 55.6757322, 55...","[12.15372974969827, 12.5262801, 12.5619711, 12...",Friday,2024-02-08,...,"[6, 6, 6, 6, 6, 6, 6, 6, 6]","[9, 9, 9, 9, 9, 9, 9, 9, 9]","[1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[1, 0, 0, 0, 0, 0, 0]",0.333333,0.200000,0.564024,train
3,"[1, 2, 9, 4, 5, 6, 7, 8]","[1, 2, 14, 156, 4, 5, 6, 7]",1255.0,"[14.640018023192697, 1.400306417188586, 0.4137...","[14.640018023192697, 1.400306417188586, 0.4137...",11.0,"[55.64641920439919, 55.6669147, 55.671473, 55....","[12.15372974969827, 12.5262801, 12.561174, 12....",Friday,2024-02-22,...,"[11, 11, 11, 11, 11, 11, 11, 11]","[8, 8, 8, 8, 8, 8, 8, 8]","[1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[1, 0, 0, 0, 0, 0, 0]",0.250000,0.200000,0.559459,train
4,"[1, 1, 2, 9, 3, 4, 5, 6, 7, 10, 8]","[1, 1, 2, 14, 3, 156, 4, 5, 6, 3769, 7]",1255.0,"[0.0, 14.640018023192697, 1.400306417188586, 0...","[0.0, 14.640018023192697, 1.400306417188586, 0...",11.0,"[55.64641920439919, 55.64641920439919, 55.6669...","[12.15372974969827, 12.15372974969827, 12.5262...",Friday,2024-02-29,...,"[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11]","[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[1, 0, 0, 0, 0, 0, 0]",0.200000,0.250000,0.554858,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17662,"[8656, 9169, 9170, 9180, 9177, 9171, 9174, 917...","[4315, 9711, 9715, 9717, 9738, 9712, 9714, 971...",1555.0,"[10.550783573784587, 0.025338060555759063, 0.1...","[10.649090169527625, 0.036295189300920946, 0.0...",9.0,"[56.0667627, 56.159131, 56.15884339999999, 56....","[9.986944, 10.204122, 10.2037157, 10.1991044, ...",Wednesday,2024-05-21,...,"[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, ...","[20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 2...","[387, 387, 387, 387, 387, 387, 387, 387, 387, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],...","[0, 0, 0, 0, 0, 0, 1]",1.000000,1.000000,0.542148,train
17663,"[8656, 9169, 9170, 9183, 9180, 9171, 9174, 917...","[4315, 9711, 9715, 9710, 9717, 9712, 9714, 971...",1555.0,"[10.550783573784587, 0.025338060555759063, 0.2...","[

## List of features

routes



experience_feature
len_feature
driver_id_feature

Total: 175


In [41]:
from torch.utils.data import Dataset, DataLoader
import torch
import random
import os
import torch.nn as nn

seed_value = 42
device = "cpu"


g = torch.Generator()

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Additional steps if using DataLoaders (to ensure reproducibility in data loading)
torch.use_deterministic_algorithms(True)

def reset_random():
    g.manual_seed(seed_value)
    torch.manual_seed(seed_value)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
    torch.use_deterministic_algorithms(True)

set_seed(42)

In [42]:
class PredictionRNN_bidirectional(nn.Module):
    def __init__(self,no_layers,vocab_size, vocab_size_driv, vocab_size_len,hidden_dim,embedding_dim,embedding_dim_driv, output_dim,additional_feature_count,drop_prob=0.5):
        super(PredictionRNN,self).__init__()

        self.output_dim = output_dim
        self.hidden_dim = hidden_dim

        self.no_layers = no_layers
        self.vocab_size = vocab_size
        self.vocab_size_driv = vocab_size_driv
        self.vocab_size_len = vocab_size_len

        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding_driv = nn.Embedding(vocab_size_driv, embedding_dim_driv)
        #lstm
        self.lstm = nn.LSTM(input_size=embedding_dim,hidden_size=self.hidden_dim,
                           num_layers=no_layers, batch_first=True)
        # self.lstm = nn.LSTM(input_size=embedding_dim,hidden_size=self.hidden_dim,
        #                    num_layers=no_layers, batch_first=True)
        # embedding_dim_driv+2
        # self.lstm = nn.LSTM(input_size=embedding_dim,hidden_size=self.hidden_dim,
        #                    num_layers=no_layers, batch_first=True)


        # dropout layer
        self.lstm = nn.LSTM(input_size=embedding_dim,hidden_size=self.hidden_dim,
                           num_layers=no_layers, batch_first=True)

        self.dropout = nn.Dropout(drop_prob)
        self.lstm = nn.LSTM(input_size=embedding_dim,
                            hidden_size=self.hidden_dim,
                            num_layers=self.no_layers,
                            batch_first=True,
                            bidirectional=True)

        # The output of the BiLSTM will have twice the hidden size due to bidirectionality
        self.lstm_output_size = self.hidden_dim * 2


        # dropout layer
        # self.fc = nn.Linear(self.hidden_dim, output_dim)
        # self.fc_static_1 = nn.Linear(embedding_dim_driv+embedding_dim_len+9, 16)
        self.fc_static_1 = nn.Linear(embedding_dim_driv+9, 128)
        # self.fc_static_2 = nn.Linear(128, 128)
        # # self.fc = nn.Linear(36, output_dim)
        # # self.fc = nn.Linear(1316, output_dim)
        self.fc = nn.Linear(128+self.lstm_output_size,128)
        # # self.fc = nn.Linear(128,128)
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.sig = nn.Sigmoid()

    def forward(self,x, is_training=True):
        batch_size = len(x['routes'])
        route_ids = x['routes'].int()

        embeds = self.embedding(route_ids)  # shape: B x S x Feature   since batch = True

        # get driver ids example
        driver_ids = x['driver_id_sorted'].int()
        experience = x['last_two_weeks_count'].int()
        len_feature = x['len'].int()
        # distance = x['distance_route'].int()
        day_of_week_feature = x['day_of_week_encoded_ext'].int()
        country_flag = x['country_flag'].int()
        location_is_depot = x['location_is_depot'].int()
        location_type_id = x['location_type_id'].int()

        embedding_driv = self.embedding_driv(driver_ids)
        # ,distance.view(batch_size, max_route_length, 1)
        # experience.view(batch_size, max_route_length, 1)

        # all_embeds = torch.concatenate((embeds, distance.view(batch_size, max_route_length, 1), location_is_depot.view(batch_size, max_route_length, 1), location_type_id.view(batch_size, max_route_length, 1)), dim=2)
        all_embeds = embeds

        # all_static = torch.concatenate((embedding_driv, embedding_len, day_of_week_feature, country_flag.view(batch_size, 1),experience.view(batch_size, 1)), dim=1)
        all_static = torch.concatenate((embedding_driv, country_flag.view(batch_size, 1),len_feature.view(batch_size, 1), day_of_week_feature), dim=1)
        # all_static = torch.concatenate((embedding_driv, day_of_week_feature), dim=1)

        lstm_out, _ = self.lstm(all_embeds)
        # lstm_out = lstm_out.contiguous().view(batch_size, self.lstm_output_size, self.hidden_dim)
        out = self.dropout(lstm_out)

        out_static = self.relu(self.fc_static_1(all_static))
        # out_static = self.dropout(self.fc_static_2(out_static))
        # out_static = out_static.unsqueeze(1)
        static_expanded = out_static.unsqueeze(1).expand(-1, lstm_out.size(1), -1)

        combined = self.fc(torch.cat((out, static_expanded), dim=2))
        # out = self.fc(torch.concatenate((out, out_static), dim=1))
        # out_static = out_static.squeeze(1).unsqueeze(1).repeat(1, 36, 1)  # New shape: [2048, 36, 64]
        # out = self.fc(torch.cat((out, out_static), dim=2))
        x = self.relu(combined)
        x = self.relu(self.layer2(x))
        # out = self.dropout(x)
        out = self.layer3(x)
        # print(out.shape)
        # sigmoid function
        sig_out = self.sig(out)

        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        # print(sig_out.shape)
        return sig_out

In [63]:
class PredictionRNN(nn.Module):
    def __init__(self,no_layers,vocab_size, vocab_size_driv, vocab_size_len,hidden_dim,embedding_dim,embedding_dim_driv, output_dim,additional_feature_count,drop_prob=0.5):
        super(PredictionRNN,self).__init__()

        self.output_dim = output_dim
        self.hidden_dim = hidden_dim

        self.no_layers = no_layers
        self.vocab_size = vocab_size
        self.vocab_size_driv = vocab_size_driv
        self.vocab_size_len = vocab_size_len

        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding_driv = nn.Embedding(vocab_size_driv, embedding_dim_driv)
        #lstm
        self.lstm = nn.LSTM(input_size=embedding_dim,hidden_size=self.hidden_dim,
                           num_layers=no_layers, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        # self.fc = nn.Linear(self.hidden_dim, output_dim)
        # self.fc_static_1 = nn.Linear(embedding_dim_driv+embedding_dim_len+9, 16)
        self.fc_static_1 = nn.Linear(embedding_dim_driv+9, 128)
        # self.fc_static_2 = nn.Linear(128, 128)
        # # self.fc = nn.Linear(36, output_dim)
        # # self.fc = nn.Linear(1316, output_dim)
        self.fc = nn.Linear(64,32) #128
        # # self.fc = nn.Linear(128,128)
        # self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.sig = nn.Sigmoid()

    def forward(self,x, is_training=True):
        batch_size = len(x['routes'])
        route_ids = x['routes'].int()

        embeds = self.embedding(route_ids)  # shape: B x S x Feature   since batch = True

        # get driver ids example
        driver_ids = x['driver_id_sorted'].int()
        experience = x['last_two_weeks_count'].int()
        len_feature = x['len'].int()
        # distance = x['distance_route'].int()
        day_of_week_feature = x['day_of_week_encoded_ext'].int()
        country_flag = x['country_flag'].int()
        location_is_depot = x['location_is_depot'].int()
        location_type_id = x['location_type_id'].int()

        embedding_driv = self.embedding_driv(driver_ids)
        # ,distance.view(batch_size, max_route_length, 1)
        # experience.view(batch_size, max_route_length, 1)

        # all_embeds = torch.concatenate((embeds, distance.view(batch_size, max_route_length, 1), location_is_depot.view(batch_size, max_route_length, 1), location_type_id.view(batch_size, max_route_length, 1)), dim=2)
        all_embeds = embeds

        # all_static = torch.concatenate((embedding_driv, embedding_len, day_of_week_feature, country_flag.view(batch_size, 1),experience.view(batch_size, 1)), dim=1)
        # country_flag.view(batch_size, 1),len_feature.view(batch_size, 1)
        # len_feature.view(batch_size, 1)
        all_static = torch.concatenate((embedding_driv, day_of_week_feature, country_flag.view(batch_size, 1), len_feature.view(batch_size, 1)), dim=1)
        # all_static = torch.concatenate((embedding_driv, day_of_week_feature), dim=1)

        lstm_out, _ = self.lstm(all_embeds)
        lstm_out = lstm_out.contiguous().view(batch_size, max_route_length, self.hidden_dim)
        out = self.dropout(lstm_out)

        out_static = self.relu(self.fc_static_1(all_static))
        # out_static = self.dropout(self.fc_static_2(out_static))
        # out_static = out_static.unsqueeze(1)
        # print(out.shape, out_static.shape)
        # out = self.fc(torch.concatenate((out, out_static), dim=1))
        out = self.fc(out)
        # out_static = out_static.squeeze(1).unsqueeze(1).repeat(1, 36, 1)  # New shape: [2048, 36, 64]
        # out = self.fc(torch.cat((out, out_static), dim=2))
        # x = self.relu(out_static)
        # x = self.relu(self.layer2(out))
        out = self.layer3(out)
        # print(out.shape)
        # sigmoid function
        sig_out = self.sig(out)

        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        # print(sig_out.shape)
        return sig_out

In [64]:
class PredictionAllFeaturesCNN(nn.Module):
    def __init__(self, no_layers, vocab_size, vocab_size_driv, embedding_dim, embedding_dim_driv, output_dim, drop_prob=0.5):
        super(PredictionAllFeaturesCNN, self).__init__()

        self.no_layers = no_layers
        self.output_dim = output_dim
        self.vocab_size = vocab_size
        self.vocab_size_driv = vocab_size_driv

        # Embedding layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding_driv = nn.Embedding(vocab_size_driv, embedding_dim_driv)

        # CNN layers
        self.conv1 = nn.Conv1d(in_channels=embedding_dim, out_channels=64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)


        # self.lstm = nn.LSTM(input_size=embedding_dim,hidden_size=self.hidden_dim,
        #                    num_layers=no_layers, batch_first=True)
        # self.lstm = nn.LSTM(input_size=embedding_dim,hidden_size=self.hidden_dim,
        #                    num_layers=no_layers, batch_first=True)
        # embedding_dim_driv+2
        # self.lstm = nn.LSTM(input_size=embedding_dim,hidden_size=self.hidden_dim,
        #                    num_layers=no_layers, batch_first=True)


        # dropout layer
        # self.lstm = nn.LSTM(input_size=embedding_dim,hidden_size=self.hidden_dim,
        #                    num_layers=no_layers, batch_first=True)

        self.dropout = nn.Dropout(drop_prob)
        # self.fc = nn.Linear(self.hidden_dim, output_dim)
        # self.fc_static_1 = nn.Linear(embedding_dim_driv+embedding_dim_len+9, 16)
        self.fc_static_1 = nn.Linear(embedding_dim_driv+9, 128)
        # self.fc_static_2 = nn.Linear(128, 128)
        # # self.fc = nn.Linear(36, output_dim)
        # # self.fc = nn.Linear(1316, output_dim)
        self.fc = nn.Linear(128+64,64)
        # # self.fc = nn.Linear(128,128)
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.sig = nn.Sigmoid()

    def forward(self, x, is_training=True):
        batch_size = len(x['routes'])
        route_ids = x['routes'].int()

        embeds = self.embedding(route_ids)  # shape: B x S x Feature   since batch = True

        # get driver ids example
        driver_ids = x['driver_id_sorted'].int()
        experience = x['last_two_weeks_count'].int()
        len_feature = x['len'].int()
        # distance = x['distance_route'].int()
        day_of_week_feature = x['day_of_week_encoded_ext'].int()
        country_flag = x['country_flag'].int()
        location_is_depot = x['location_is_depot'].int()
        location_type_id = x['location_type_id'].int()

        embedding_driv = self.embedding_driv(driver_ids)
        # ,distance.view(batch_size, max_route_length, 1)
        # experience.view(batch_size, max_route_length, 1)

        # all_embeds = torch.concatenate((embeds, distance.view(batch_size, max_route_length, 1), location_is_depot.view(batch_size, max_route_length, 1), location_type_id.view(batch_size, max_route_length, 1)), dim=2)
        all_embeds = embeds

        # all_static = torch.concatenate((embedding_driv, embedding_len, day_of_week_feature, country_flag.view(batch_size, 1),experience.view(batch_size, 1)), dim=1)
        all_static = torch.concatenate((embedding_driv, country_flag.view(batch_size, 1),len_feature.view(batch_size, 1), day_of_week_feature), dim=1)
        # all_static = torch.concatenate((embedding_driv, day_of_week_feature), dim=1)


        cnn_input = embeds.permute(0, 2, 1)  # Reshape for CNN: B x Feature x S
        cnn_out = self.conv1(cnn_input)
        cnn_out = self.relu(cnn_out)
        cnn_out = self.pool(cnn_out)
        # cnn_out = cnn_out.permute(0, 2, 1)
        cnn_out = torch.mean(cnn_out, dim=2)
        cnn_out = self.dropout(cnn_out)

        # lstm_out, _ = self.lstm(cnn_out)
        # # lstm_out = lstm_out.contiguous().view(batch_size, max_route_length, self.hidden_dim)
        # lstm_out = lstm_out.contiguous().view(batch_size, -1, self.hidden_dim)
        # out = self.dropout(lstm_out)

        out_static = self.relu(self.fc_static_1(all_static))
        # out_static = self.dropout(self.fc_static_2(out_static))
        # out_static = out_static.unsqueeze(1)

        combined = self.fc(torch.cat((cnn_out, out_static), dim=1))
        # out = self.fc(torch.concatenate((out, out_static), dim=1))
        # out_static = out_static.squeeze(1).unsqueeze(1).repeat(1, 36, 1)  # New shape: [2048, 36, 64]
        # out = self.fc(torch.cat((out, out_static), dim=2))
        x = self.relu(combined)
        # x = self.relu(self.layer2(out))
        # out = self.dropout(x)
        out = self.layer3(x)
        # print(out.shape)
        # sigmoid function
        sig_out = self.sig(out)

        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        # print(sig_out.shape)
        return sig_out

In [65]:
class PredictionAllFeaturesGRU(nn.Module):
    def __init__(self, no_layers, vocab_size, vocab_size_driv, embedding_dim, embedding_dim_driv, hidden_dim, output_dim, drop_prob=0.5):
        super().__init__()

        self.no_layers = no_layers
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.vocab_size = vocab_size
        self.vocab_size_driv = vocab_size_driv

        # Embedding layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding_driv = nn.Embedding(vocab_size_driv, embedding_dim_driv)

        # GRU layer for route features
        self.gru = nn.GRU(input_size=embedding_dim,
                          hidden_size=hidden_dim,
                          num_layers=no_layers,
                          batch_first=True,
                          dropout=drop_prob if no_layers > 1 else 0)

        self.dropout = nn.Dropout(drop_prob)
        self.fc_static_1 = nn.Linear(embedding_dim_driv+8, 128)
        self.fc = nn.Linear(128+hidden_dim, 128)
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(128, 1)
        self.relu = nn.ReLU()
        self.sig = nn.Sigmoid()

    def forward(self, x):
        batch_size = len(x['routes'])
        route_ids = x['routes'].int()

        embeds = self.embedding(route_ids)  # shape: B x S x Feature

        # Static features
        driver_ids = x['driver_id_sorted'].int()
        experience = x['last_two_weeks_count'].int()
        len_feature = x['len'].int()
        day_of_week_feature = x['day_of_week_encoded_ext'].int()
        country_flag = x['country_flag'].int()
        location_is_depot = x['location_is_depot'].int()
        location_type_id = x['location_type_id'].int()

        embedding_driv = self.embedding_driv(driver_ids)

        all_static = torch.cat((embedding_driv, country_flag.view(batch_size, 1),
                                len_feature.view(batch_size, 1), day_of_week_feature), dim=1)

        # Process route features with GRU
        gru_out, _ = self.gru(embeds)
        gru_out = gru_out[:, -1, :]  # Take the last output of the sequence
        gru_out = self.dropout(gru_out)

        out_static = self.relu(self.fc_static_1(all_static))

        combined = self.fc(torch.cat((gru_out, out_static), dim=1))
        x = self.relu(combined)
        out = self.layer3(x)

        sig_out = self.sig(out)
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1]  # get last batch of labels

        return sig_out

In [66]:
import math


class RunningNorm:
    def __init__(self, dim=0):
        self.dim = dim
        self.n = 0
        self.mean = 0
        self.var = 0

    def update(self, x):
        self.n += 1
        if self.n == 1:
            self.mean = x
            self.var = torch.zeros_like(x)
        else:
            new_mean = self.mean + (x - self.mean) / self.n
            self.var = (self.var * (self.n - 1) + (x - self.mean) * (x - new_mean)) / self.n
            self.mean = new_mean

    def normalize(self, x):
        if self.n == 0:
            return x
        return (x - self.mean) / (torch.sqrt(self.var) + 1e-8)

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=35):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x.shape should be (batch_size, sequence_length, d_model)
        sequence_length = x.size(1)
        x = x + self.pe[:, :sequence_length, :]
        return x

class SinusoidalPosEmb(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, x):
        device = x.device
        half_dim = self.dim // 2
        emb = math.log(10000) / (half_dim - 1)
        emb = torch.exp(torch.arange(half_dim, device=device) * -emb)
        emb = x[:, None] * emb[None, :]
        emb = torch.cat((emb.sin(), emb.cos()), dim=-1)
        return emb


# "Encoder-Only" Style Transformer
class NanoTransformer(nn.Module):
    """
    This class implements a simplified Transformer model for sequence classification.
    It uses an embedding layer for tokens, sinusoidal positional embeddings,
    a single Transformer block, and a final linear layer for prediction.

    Args:
      num_emb: The number of unique tokens in the vocabulary.
      output_size: The size of the output layer (number of classes).
      hidden_size: The dimension of the hidden layer in the Transformer block (default: 128).
      num_heads: The number of heads in the multi-head attention layer (default: 4).
    """
    def __init__(self, embedding_dim, vocab_size, embedding_dim_driv, vocab_size_driv, hidden_size=64, num_heads=4):
        super(NanoTransformer, self).__init__()

        # Create an embedding for each token
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding_driv = nn.Embedding(vocab_size_driv, embedding_dim_driv)
        self.embedding.weight.data = 0.001 * self.embedding.weight.data

        self.hidden_size = hidden_size
        self.embedding_dim_driv = embedding_dim_driv
        self.vocab_size_driv = vocab_size_driv

        self.pos_emb = SinusoidalPosEmb(hidden_size)

        self.multihead_attn1 = nn.MultiheadAttention(hidden_size, num_heads=num_heads, batch_first=True)
        self.layer_norm = nn.LayerNorm(hidden_size)
        self.fc_static_1 = nn.Linear(embedding_dim_driv+9, 128)
        self.layer3 = nn.Linear(128, 1)
        self.fc = nn.Linear(hidden_size+128, 64)
        self.layer3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.sig = nn.Sigmoid()



    def forward(self, x, is_training=True):
        bs = len(x['routes'])
        l = len(x['routes'][0])
        route_ids = x['routes'].int()
        input_embs = self.embedding(route_ids)  # shape: B x S x Feature   since batch = True

        driver_ids = x['driver_id_sorted'].int()
        experience = x['last_two_weeks_count'].float()
        len_feature = x['len'].float()
        day_of_week_feature = x['day_of_week_encoded_ext'].int()
        country_flag = x['country_flag'].int()
        location_is_depot = x['location_is_depot'].int()
        location_type_id = x['location_type_id'].int()

        embedding_driv = self.embedding_driv(driver_ids)

        all_static = torch.cat((embedding_driv, country_flag.view(bs, 1),
                                len_feature.view(bs, 1), day_of_week_feature), dim=1)


        seq_indx = torch.arange(l)
        pos_emb = self.pos_emb(seq_indx).reshape(1, l, -1).expand(bs, l, -1)
        embs = input_embs + pos_emb
        output1, attn_map1 = self.multihead_attn1(embs, embs, embs)
        output = self.layer_norm(embs + output1)

        out_static = self.relu(self.fc_static_1(all_static))
        out_static = out_static.unsqueeze(1).expand(-1, l, -1)

        out = self.fc(torch.cat((output, out_static), dim=2))
        out = self.layer3(out)
        sig_out = self.sig(out).squeeze(-1)
        final_out = torch.mean(sig_out, dim=1)
        # seq_indx = torch.arange(l)
        # pos_emb = self.pos_emb(seq_indx).reshape(1, l, -1).expand(bs, l, -1)
        # embs = input_embs + pos_emb
        # output, attn_map = self.multihead_attn(embs, embs, embs)
        #
        # out_static = self.bn1(self.relu(self.fc_static_1(all_static)))
        #
        # out_static = out_static.squeeze(1).unsqueeze(1).repeat(1, 36, 1)  # New shape: [2048, 36, 64]
        # out = self.fc(torch.cat((output, out_static), dim=2))

        # print(out.shape)
        # out = self.layer3(out)
        # sig_out = self.sig(out)
        # print(sig_out.shape)
        # sig_out = sig_out.view(bs, -1)
        # print(sig_out.shape)
        # sig_out = sig_out[:, -1] # get last batch of labels
        # print(sig_out.shape)
        # raise 'ere'
        return final_out

In [80]:
def get_model():
    no_layers = 2
    vocab_size = 10547 #extra 1 for padding
    vocab_size_driv = 317
    vocab_size_len = 37 #extra 1 for padding
    embedding_dim = 128 #was 64
    embedding_dim_driv = 32 #was 64
    # embedding_dim_len = 2 #was 64
    output_dim = 1
    hidden_dim = 64 #was 64

    # model = PredictionAllFeaturesCNN(no_layers, vocab_size, vocab_size_driv, embedding_dim, embedding_dim_driv, output_dim, drop_prob=0.5)
    model = PredictionRNN(no_layers,vocab_size, vocab_size_driv, vocab_size_len, hidden_dim,embedding_dim, embedding_dim_driv, output_dim,len(drivers_dic),drop_prob=0.5)
    # model = NanoTransformer(embedding_dim, vocab_size, embedding_dim_driv, vocab_size_driv)
    model.train()
    print(model)
    return model

In [81]:
# function to predict accuracy
# def acc(pred,label):
#     pred = torch.round(pred.squeeze())
#     return torch.sum(pred == label.squeeze()).item()

def rmse(pred, label):
    return torch.sqrt(torch.mean((pred.squeeze() - label.squeeze())**2))

# def get_precision(pred, label):
#     pred = torch.round(pred.squeeze())
#     true_positive = torch.sum((pred == 1) & (label.squeeze() == 1)).item()
#     false_positive = torch.sum((pred == 1) & (label.squeeze() == 0)).item()
#
#     if true_positive + false_positive == 0:
#         return 0.0, true_positive, false_positive  # Handle the case where there are no predicted positives
#
#     precision_value = true_positive / (true_positive + false_positive)
#     return precision_value, true_positive, false_positive

In [82]:
RUN_NAME = 'first_run_5'
def train_model(data, model, epochs = 20):
    train_loader, valid_loader = data
    lr=0.001

    criterion = nn.MSELoss()

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    clip = 5
    valid_loss_min = np.Inf
    # train for some number of epochs
    epoch_tr_loss,epoch_vl_loss = [],[]
    epoch_tr_acc,epoch_vl_acc = [],[]
    epoch_tr_precision,epoch_vl_precision = [],[]
    # wandb.init(project='Route_classification', name=f'{run_name}')

    for epoch in range(epochs):
        train_losses = []
        train_rmse = 0.0
        model.train()
        for inputs, labels in train_loader:
            # print(labels)

            model.zero_grad()
            # print(inputs.shape, h[0].shape, h[1].shape)
            output = model(inputs)

            # calculate the loss and perform backprop
            # print(output, labels)
            # print(output.shape, labels.shape)
            loss = criterion(output.view(-1), labels.float())
            loss.backward()
            train_losses.append(loss.item())
            #`clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()
        val_losses = []
        model.eval()
        with torch.no_grad():  # Disable gradient computation for validation
            for inputs, labels in valid_loader:
                output = model(inputs, is_training=False)  # Set is_training to False for validation
                val_loss = criterion(output.view(-1), labels.float())
                val_losses.append(val_loss.item())

        epoch_train_loss = np.mean(train_losses)
        epoch_val_loss = np.mean(val_losses)

        epoch_tr_loss.append(epoch_train_loss)
        epoch_vl_loss.append(epoch_val_loss)
        print(f'Epoch {epoch+1}')
        print(f'train_loss : {epoch_train_loss} val_loss : {epoch_val_loss}')

        if epoch_val_loss <= valid_loss_min:
            torch.save(model.state_dict(), f'{RUN_NAME}.pt')
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,epoch_val_loss))
            valid_loss_min = epoch_val_loss
        print(25*'==')

In [83]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score, log_loss, brier_score_loss, confusion_matrix
)
import matplotlib.pyplot as plt

def get_stats(model, data):
    model.eval()
    y_pred = []
    y = []
    all_routes = []
    with torch.no_grad():  # Disable gradient computation
        for inputs, labels in data:
            y.extend(labels)
            results = model(inputs, is_training=False).detach()  # Set is_training to False
            y_pred.extend(results)
            all_routes.extend(inputs['routes'].cpu().numpy())

    y_true = np.array(y)
    y_pred_proba = np.array(y_pred)
    y_pred = (y_pred_proba > 0.5).astype(np.float32)

    acc = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_pred)
    average_precision = average_precision_score(y_true, y_pred)
    quadratic_loss = log_loss(y_true, y_pred_proba)
    brier_score = brier_score_loss(y_true, y_pred_proba)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    return {
        "acc": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "roc_auc": roc_auc,
        "average_precision": average_precision,
        "quadratic_loss": quadratic_loss,
        "brier_score": brier_score,
        "true_positive": tp,
        "false_positive": fp,
        "true_negative": tn,
        "false_negative": fn
    }


In [84]:
def get_stats(model, data):
    model.eval()
    y_pred = []
    y = []
    all_inputs = []
    with torch.no_grad():
        for inputs, labels in data:
            y.extend(labels.cpu().numpy())
            results = model(inputs, is_training=False).detach()
            y_pred.extend(results.cpu().numpy())
            all_inputs.append({k: v.cpu().numpy() for k, v in inputs.items()})

    y_true = np.array(y)
    y_pred_proba = np.array(y_pred)
    y_pred_binary = (y_pred_proba > 0.5).astype(np.float32)

    # Calculate metrics
    acc = accuracy_score(y_true, y_pred_binary)
    precision = precision_score(y_true, y_pred_binary)
    recall = recall_score(y_true, y_pred_binary)
    f1 = f1_score(y_true, y_pred_binary)
    roc_auc = roc_auc_score(y_true, y_pred_proba)
    average_precision = average_precision_score(y_true, y_pred_proba)
    quadratic_loss = log_loss(y_true, y_pred_proba)
    brier_score = brier_score_loss(y_true, y_pred_proba)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred_binary).ravel()

    # Combine all inputs into a single dictionary
    combined_inputs = {k: np.concatenate([d[k] for d in all_inputs]) for k in all_inputs[0].keys()}

    return {
        "metrics": {
            "acc": acc,
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "roc_auc": roc_auc,
            "average_precision": average_precision,
            "quadratic_loss": quadratic_loss,
            "brier_score": brier_score,
            "true_positive": tp,
            "false_positive": fp,
            "true_negative": tn,
            "false_negative": fn
        },
        "predictions": {
            "y_true": y_true,
            "y_pred_proba": y_pred_proba,
            "y_pred_binary": y_pred_binary,
            "inputs": combined_inputs
        }
    }

def categorize_predictions(df, stats):
    y_true = stats['predictions']['y_true']
    y_pred_binary = stats['predictions']['y_pred_binary']
    y_pred_proba = stats['predictions']['y_pred_proba']

    df = df.copy()
    df['prediction_category'] = 'Unknown'
    df.loc[(y_true == 1) & (y_pred_binary == 1), 'prediction_category'] = 'True Positive'
    df.loc[(y_true == 0) & (y_pred_binary == 1), 'prediction_category'] = 'False Positive'
    df.loc[(y_true == 1) & (y_pred_binary == 0), 'prediction_category'] = 'False Negative'
    df.loc[(y_true == 0) & (y_pred_binary == 0), 'prediction_category'] = 'True Negative'
    df['predictive_probability'] = y_pred_proba

    # Add other features from the inputs if needed
    for feature, values in stats['predictions']['inputs'].items():
        if feature not in df.columns:
            df[feature] = values

    return df

In [86]:
from sklearn.model_selection import KFold, cross_val_score
set_seed(42)

FEATURE_COLUMNS = ['routes', 'driver_id_feature', 'driver_id_sorted','last_two_weeks_count', 'len', 'day_of_week_feature', 'day_of_week_encoded_ext', 'country_flag', 'location_is_depot', 'location_type_id']


class DataFrameDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        item = {}
        for col in FEATURE_COLUMNS:
            feature = self.df[col].iloc[idx]
            if isinstance(feature, list):
                feature = padding_([feature], max_route_length)[0]
            item[col] = feature
        item['label'] = self.df['label'].iloc[idx]
        return item, item['label']

def get_data_loaders(train_df, val_df, test_df):
    # create Tensor datasets
    train_data = DataFrameDataset(train_df)
    val_data = DataFrameDataset(val_df)
    test_data = DataFrameDataset(test_df)

    # dataloaders
    batch_size = 2048

    # make sure to SHUFFLE your data
    train_loader = DataLoader(train_data, shuffle=False, batch_size=batch_size, num_workers=0, generator=g, worker_init_fn=seed_worker)
    val_loader = DataLoader(val_data, shuffle=False, batch_size=batch_size, num_workers=0, generator=g, worker_init_fn=seed_worker)
    test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size,num_workers=0, generator=g, worker_init_fn=seed_worker)
    return train_loader, val_loader, test_loader

train_df = final_routes_split[final_routes_split['split'] == 'train'].drop('split', axis=1)
val_df = final_routes_split[final_routes_split['split'] == 'val'].drop('split', axis=1)
test_df = final_routes_split[final_routes_split['split'] == 'test'].drop('split', axis=1)

print(f'len(train_df) = {len(train_df)}, len(val_df) = {len(val_df)}, len(test_df) = {len(test_df)}')

train_loader, val_loader, test_loader = get_data_loaders(train_df, val_df, test_df)
model = get_model()
model.train()

# Train the model
train_model((train_loader, val_loader), model, epochs=40)

# Load the best model
model = get_model()
model.load_state_dict(torch.load(f'{RUN_NAME}.pt'))

# Evaluate on test set
model.eval()
test_stats = get_stats(model, test_loader)

print("Metrics:", test_stats['metrics'])

categorized_df = categorize_predictions(test_df, test_stats)

len(train_df) = 13823, len(val_df) = 2188, len(test_df) = 1656
PredictionRNN(
  (embedding): Embedding(10547, 128)
  (embedding_driv): Embedding(317, 32)
  (lstm): LSTM(128, 64, num_layers=2, batch_first=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc_static_1): Linear(in_features=41, out_features=128, bias=True)
  (fc): Linear(in_features=64, out_features=32, bias=True)
  (layer3): Linear(in_features=32, out_features=1, bias=True)
  (relu): ReLU()
  (sig): Sigmoid()
)
Epoch 1
train_loss : 0.25870091787406374 val_loss : 0.2588515728712082
Validation loss decreased (inf --> 0.258852).  Saving model ...
Epoch 2
train_loss : 0.25046323026929584 val_loss : 0.24678972363471985
Validation loss decreased (0.258852 --> 0.246790).  Saving model ...
Epoch 3
train_loss : 0.23781342378684453 val_loss : 0.22301308065652847
Validation loss decreased (0.246790 --> 0.223013).  Saving model ...
Epoch 4
train_loss : 0.21506014040538243 val_loss : 0.1890869364142418
Validation loss decreased (0.22

In [ ]:
ha_tp_routes = test_df_group_HA[test_df_group_HA['prediction_category'] == 'False Positive']['routes'].tolist()
lstm_tp_routes = categorized_df[categorized_df['prediction_category'] == 'False Positive']['routes'].tolist()

print(len(categorized_df[categorized_df['prediction_category'] == 'False Negative']['routes']), len(categorized_df[categorized_df['prediction_category'] == 'False Negative']['routes']))
# Convert lists to sets of tuples for comparison
ha_tp_routes_set = set(tuple(route) for route in ha_tp_routes)
lstm_tp_routes_set = set(tuple(route) for route in lstm_tp_routes)
common_routes = ha_tp_routes_set.intersection(lstm_tp_routes_set)

# Find non-overlapping routes
ha_only_routes = ha_tp_routes_set - lstm_tp_routes_set
lstm_only_routes = lstm_tp_routes_set - ha_tp_routes_set

# Convert back to lists for display
common_routes = [list(route) for route in common_routes]
ha_only_routes = [list(route) for route in ha_only_routes]
lstm_only_routes = [list(route) for route in lstm_only_routes]

print("Routes unique to HA model:", len(ha_only_routes), len(common_routes))
print("Sample of HA-only routes:", ha_only_routes[:5])

print("\nRoutes unique to LSTM model:", len(lstm_only_routes))
print("Sample of LSTM-only routes:", lstm_only_routes[:5])
print(len(common_routes))

CNN:
1632: 793 label0,
TP: 54 561 benchmark
    75 561 model
FP: 88 153 benchmark
    49 153 model
TN: 49 494 benchmark
    86 494 model
FN: 73 141 benchmark
    53 141 model

0.15
TP 80 556
   60 556
FP: 112 149
    36 149
TN: 36 491
    110 491
FN: 60 156 benchmark
    79 156 benchmark

0.3
TP: 45 567 benchmark
    64 567 model
FP: 82 167 benchmark
    56 167 model
TN: 55 505 benchmark
    81 505 model
FN: 64 155 benchmark
    44 155 model




a = 561+494, b = 54+49, c = 75+86, d = 153+141

# print(categorized_df['routes'])

In [ ]:
# Example of calculating the mcnemar test
from mlxtend.evaluate import mcnemar
# define contingency table
table = np.array([[1055, 103],
		 [161, 294]])
# calculate mcnemar test
result = mcnemar(table)
print(result)
# summarize the finding
# print('statistic=%.3f, p-value=%.10f' % (result.statistic, result.pvalue))
# interpret the p-value

In [ ]:
!pip install mlxtend

In [ ]:
print(categorized_df['routes'])

In [ ]:
len(test_df[test_df['label']==1])

In [ ]:
def get_mean_stat(stats, stat_name):
    # Calculate mean for regression stats
    arr = np.array([item[stat_name] for item in stats])
    return arr.mean()
print('mse:', get_mean_stat(stats, 'mse'))
print('rmse:', get_mean_stat(stats, 'rmse'))
print('mae:', get_mean_stat(stats, 'mae'))
print('r2:', get_mean_stat(stats, 'r2'))

# If you want to visualize the distribution of predictions vs actual values
plt.figure(figsize=(10, 6))
for stat in stats:
    plt.scatter(stat['y'], stat['y_pred'], alpha=0.5)
plt.plot([0, 1], [0, 1], 'r--')
plt.xlabel('True Values')
plt.ylabel('Predicted Values')
plt.title('Predicted vs True Values')
plt.show()

# If you want to visualize the residuals
plt.figure(figsize=(10, 6))
for stat in stats:
    residuals = np.array(stat['y_pred']) - np.array(stat['y'])
    plt.scatter(stat['y'], residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('True Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.show()

In [ ]:
# from sklearn.model_selection import train_test_split
#
# reset_random()
# X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, random_state=seed_value)
# train_loader, valid_loader = get_data_loaders(X_train, y_train, X_val, y_val)
#
# model = get_model()
# # summary(model, input_size=(1,9))
# model.train()
#
# train_model((train_loader, valid_loader), model, epochs=50)
#
# get_stats(model, valid_loader)

In [ ]:
get_stats(model, valid_loader)
# def get_mean_stat(stats, stat_name) -> float:
#     return np.array([item[stat_name] for item in stats]).mean()
#
# print('acc:', get_mean_stat(stats, 'acc'))
# print('precision:', get_mean_stat(stats, 'precision'))
# print('recall:', get_mean_stat(stats, 'recall'))
# print('f1:', get_mean_stat(stats, 'f1'))
# print('roc_auc:', get_mean_stat(stats, 'roc_auc'))
# print('average_precision:', get_mean_stat(stats, 'average_precision'))

In [ ]:
print(stats)

In [ ]:
print(model.fc.weight)

In [ ]:
test_df[test_df['label'] == 0]